# Metabolomics Data Analysis - FT-ICR-MS-Lisboa

This notebook provides a template for standard metabolomics data analysis whether or not you already annoted your data with names and/or chemical formulas using Metaboscape.

This allows you to annotate, treat, see common and exclusive metabolites, do multivariate unsupervised and supervised statistical analysis, do univariate statistical analysis, produce Van Krevelen diagrams, Kendrick Mass Defect plots and chemical composition series. It also has compound finders that allow you to search for specific compounds using their names, chemical formulas, _m/z_'s or neutral masses.

A wide variety of data pre-treatments is provided, but you don't have to use all of them. Some of the functions may seem intimidating, but they are easily costumizable and instructions to do that are provided.

**Warning**: Reading files is very dependent on the format of the file, you'll have to see and adapt that part and not just press run all on the notebook.

**Please inform us if you think something would be useful here that is not here already, other things you would like to be able to do or if something breaks spectacularly. As with 1.0 version, 2.0 version is likely to have some minor bugs and errors for specific cases that we will correct in future versions.**

### References

A lot of materials (Python functions) from the following paper was used for this pipeline (furthermore, BinSim analysis as described in the paper is also done here in the notebook):

- Traquete, F.; Luz, J.; Cordeiro, C.; Sousa Silva, M.; Ferreira, A.E.N. Binary Simplification as an Effective Tool in Metabolomics Data Analysis. _Metabolites_ 2021, 11, doi:10.3390/metabo11110788.

Apart from this, 3 specific Python libraries that are used should also be mentioned:

- Metabolinks (our in-house Python library) - https://zenodo.org/record/5336951#.Y-TbpnbP1D8.
- UpSetPlot (to make the UpSetPlots only) - https://pypi.org/project/UpSetPlot/0.8.0/.
- Pyvenn (to make the Venn Diagrams only) - https://github.com/tctianchi/pyvenn.

Finally, the most common Python libraries that are used here are pandas, numpy, scipy, scikit-learn, matplotlib and seaborn.

# Table of Contents <a class="anchor" id="toc"></a>

- **[Step 0: Installing Metabolinks and other packages](#step-0)**
- **[Step 1: Upload your data](#step-1)**
  - **[Step 1.1: Define your groups and see data characterization](#step-1_1)**
  - **[Step 1.2: Annotate with Database(s)](#step-1_2)**
  - **[Step 1.3: De-duplicating annotations](#step-1_3)**  
- **[Step 2: Basic processing and pre-treatment](#step-2)**
- **[Step 3: Find Common and Exclusive metabolites between the groups](#step-3)**
- **[Step 4: Unsupervised Statistical Analysis (PCA and HCA)](#step-4)**
- **[Step 5: Supervised Statistical Analysis (Random Forest and PLS-DA)](#step-5)**
  - **[Step 5.1: Random Forest](#step-5_1)**
  - **[Step 5.2: PLS-DA (Partial Least Squares - Discriminant Analysis)](#step-5_2)**
  - **[Step 5.3: XGBoost (eXtreme Gradient Boosting)](#step-5_3)**
- **[Step 6: Univariate Analysis and Fold-Change Analysis](#step-6)**
  - **[Step 6.1: 2-class Univariate Statistical Analysis](#step-6_1)**
  - **[Step 6.2: Multi-class Univariate Statistical Analysis](#step-6_2)**
- **[Step 7: Make Van Krevelen Diagrams, Kendrick Mass Defect Plots and Chemical Composition series for your samples](#step-7)**
- **[Step 8: Pathways Assignment of HMDB Annotated Metabolites](#step-8)**
- **[Step 9: KEGG Colour Mapping](#step-9)**
- **[Step 10: BinSim Specific Analysis](#step-10)**
- **[Step 11: Find Specific Compounds](#step-11)**

### Some common needed imports

In [ ]:
import pandas as pd
import numpy as np

import re
import math

from tqdm import tqdm

import itertools

import scipy.spatial.distance as dist
import scipy.cluster.hierarchy as hier
import scipy.stats as stats

import sklearn.ensemble as skensemble
from sklearn.metrics import (roc_auc_score, roc_curve, auc,
                             f1_score, precision_score, recall_score)
import sklearn.model_selection
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
from matplotlib import ticker
from matplotlib import cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colors import TwoSlopeNorm

import seaborn as sns
import pickle

import upsetplot
from upsetplot import from_contents
from upsetplot import UpSet

# Our Python package
import metabolinks as mtl
import metabolinks.transformations as transf

# elips.py file (has to be in the same folder)
from elips import plot_confidence_ellipse
# venn.py file
import venn as venn
# metanalysis_standard.py file
import metanalysis_standard as metsta
# some functions from interface_aux_functions
from interface_aux_functions import create_element_counts
# multianalysis.py file (has to be in the same folder)
from multianalysis import p_adjust_bh, fit_PLSDA_model, _calculate_vips, _generate_y_PLSDA

# Step 0: Installing Metabolinks and other packages<a class="anchor" id="step-0"></a>

**Back to [Table of Contents](#toc)**

**Metabolinks and UpSetPlot**

- Open 'Command Line' or 'Linha de comandos' on your pc.
- Run the line 'pip install metabolinks'.
- Run the line 'pip install UpSetPlot'.
- Run the line 'conda install -c conda-forge py-xgboost'.
- Restart jupyter.

**pyvenn**

- Go to https://github.com/tctianchi/pyvenn/blob/master/venn.py.
- Over 'Raw' (see where 'Watch' is on the upper left and move look down), click 'Save link as...'.
- Save it on the same folder you have the rest of the files.


# Step 1: Upload your data <a class="anchor" id="step-1"></a>

**Back to [Table of Contents](#toc)**

(**Warning**: Depending on the format of your file, this can be different!)

This is made for csv and excel files extracted from MetaboScape but can also work with different data tables. Default is csv, excel is last block of text (remove # if you want to use it). If you have a target in the first row of your data, make `target_in_file` True.

Select if the masses in the first column of your data (idx_masses) are Neutral masses ('Neutral'), m/z values obtained from positive ionization mode ('Positive') or negative ionization mode ('Negative') or if the values cannot be interpreted as masses ('None' - this will not allow you to perform data annotation downstream).

What does this do?
- Produces a pandas DataFrame with your spectral data
- Renames the columns to obtain "clean" sample names (if necessary)
- Replaces 0 values with numpy null objects (nan). This is necessary to know how many metabolites each sample has and to allow further data processing.

In [ ]:
# For .csv or .xslsx from MetaboScape (they still might have some differences, for example in the headers)
# Example

filename = '5yeasts_notnorm.csv' # Name of your file
#filename = 'Farinhas de Grilo (2).xlsx'
## Indicate if the file read includes the target (sample classes) in its first row
target_in_file = False
idx_masses = 'Neutral' # 'Neutral' (neutral masses), 'Positive' (Obtained from ESI+), 'Negative' (Obtained from ESI-)
# or 'None' (mass column cannot be inferred)


# Samples names frequently have 00000. Use this code to make them 'cleaner'.
# If not needed just skip this part (use #) when the function is applied
def renamer(colname):
    # Util to optionally remove all those 00000 from sample names
    return ''.join(colname.split('00000'))

# If it is a .csv file
if filename.endswith('csv'):
    if target_in_file: # If you have the target in the file
        file = pd.read_csv(filename, header=[0,1])
        colnames = [renamer(i) for i in file.columns.get_level_values(0)]
        target_file = dict(zip(colnames, file.columns.get_level_values(1)))
        file.columns = colnames

    else: # If you do not have the target in the file
        file = pd.read_csv(filename)
        file.columns = [renamer(i) for i in file.columns]

# If it is a .xlsx file
elif filename.endswith('xlsx'):
    if target_in_file: # If you have the target in the file
        file = pd.read_excel(filename, header=[0,1])
        colnames = [renamer(i) for i in file.columns.get_level_values(0)]
        target_file = dict(zip(colnames, file.columns.get_level_values(1)))
        file.columns = colnames

    else: # If you do not have the target in the file
        file = pd.read_excel(filename)
        file.columns = [renamer(i) for i in file.columns]    
else:
    raise ValueError('Provided file is not an Excel or a csv file.')

 # Important for database match - calculating Neutral Mass / Probable m/z column if possible
if idx_masses != 'None':
    # If the masses in the index are Neutral
    if idx_masses == 'Neutral':
        file.insert(1, 'Neutral Mass', file[file.columns[0]].str.replace('Da', '').astype('float'))
    # If the masses are m/z values obtained in Positive Ionization Mode
    if idx_masses == 'Positive':
        file.insert(1, 'Probable m/z', file[file.columns[0]].astype('str').str.replace('Da', '').astype('float'))
    # If the masses are m/z values obtained in Negative Ionization Mode
    elif idx_masses == 'Negative':
        file.insert(1, 'Probable m/z', file[file.columns[0]].astype('str').str.replace('Da', '').astype('float'))

file = file.set_index(file.columns[0])
file.index.name = 'Bucket label'

# Select the column with the masses to compare
if idx_masses == 'Neutral':
    mass_val_col = 'Neutral Mass'
else:
    mass_val_col = 'Probable m/z'

# Replaces zeros with numpy nans. Essential for data processing
file = file.replace({0.0:np.nan})

In [ ]:
# Identify metadata columns
metadata_cols = ['Neutral Mass', 'Probable m/z', 'm/z', 'Formula', 'Name']

# Identify sample columns - automatic based on metadata columns selected
sample_cols = []
for c in file.columns:
    if c not in metadata_cols:
        sample_cols.append(c)
print(sample_cols)

If you are working with a regression problem (labels are numerical values in a scale), set the regression parametre to True. Otherwise, it is assumed that you are working with a classification problem (label are categories)

# Step 1.1: Define your groups and see data characterization <a class="anchor" id="step-1_1"></a>

**Back to [Table of Contents](#toc)**

Here, we will define our groups (yeast strains in our example), so that we can obtain merged dataframes for each one of them and define colours to help us visualise them in later figures.

If your read file already included the target, we will use that one, confirm if it is correct. If it included but you want to change it, write the target you want and put `target_overwrite` to True.

In [ ]:
# Insert your groups in the "target_temp" list by the exact order in which they appear in the dataframe.
# Remember that each group must appear as many times as there are samples belonging to that group.
# If your target was in the file you provided, you do not need to update this list
target_temp = ['WT', 'WT', 'WT', 'dGRE3', 'dGRE3', 'dGRE3', 'dENO1', 'dENO1', 'dENO1', 'dGLO1', 'dGLO1', 'dGLO1',
          'dGLO2', 'dGLO2', 'dGLO2']
#target_temp = ['Aqua', 'Aqua', 'Aqua', 'Org', 'Org', 'Org']

target_overwrite = False # Change in case you want to overwrite the target in file given

target = target_temp # Default target

if target_in_file:
    target = [target_file[s] for s in sample_cols] # Update to target present in file provided

    if target_overwrite:
        target = target_temp # Overwrite target to the one defined by you


# See if the classes are those that you want
classes = list(pd.unique(target))
classes

#### Colors for plots to ensure consistency

Play around with colours to get the ones you want. 

- Colormaps like the tab10 used below: https://matplotlib.org/stable/tutorials/colors/colormaps.html
- List of names of individual colours: https://matplotlib.org/stable/gallery/color/named_colors.html

In [ ]:
# customize label colors

colours = sns.color_palette('tab10', 10) # Only room for 10 classes in this case, choose your colours
#colours = ('coral', 'turquoise', 'gold', 'indigo', 'lightgreen') # Example for using named colours
ordered_labels = classes # Put the classes, you can choose the order

label_colours = {lbl: c for lbl, c in zip(ordered_labels, colours)}
sample_colours = [label_colours[lbl] for lbl in target]

# See the colours for each class
sns.palplot(label_colours.values())
new_ticks = plt.xticks(range(len(ordered_labels)), ordered_labels)

Initial Filtering of the data if you want to do it. This function is explained and used in more detail after in **step 2**.

In [ ]:
min_samples_feature_appear = 2 # Select the minimum number of samples a feature must appear to be kept in the dataset

meta_cols = [i for i in file.columns if i not in sample_cols]
temp = metsta.basic_feat_filtering(file[sample_cols].T, target=None,
                            filt_method='total_samples', # Method
                            filt_kw=min_samples_feature_appear) # Make a sample have to appear
file = pd.concat((file[meta_cols].reindex(temp.columns), temp.T), axis=1)

In [ ]:
file

In [ ]:
# See data characterization
data_characteristics = metsta.characterize_data(file[sample_cols].T, target=target)
data_characteristics

**Remove the reference feature if you have already normalized your data by the reference feature previously**

In [ ]:
ref_feat_lbl = ''

if ref_feat_lbl != '':
    file = file.drop(index=ref_feat_lbl)

# Step 1.2: Annotate with Database(s) <a class="anchor" id="step-1_2"></a>

**Back to [Table of Contents](#toc)**

This is where you upload your database(s) to annotate your experimental dataset. 

Make sure that your file contains at least the following columns:
- One with the databases accession label. This will serve as the index. In case you are a masochist and have opted to create your own database, make sure you give each compound an accession label (eg. DB0001, DB0002, DB0003, etc.)
- One with the compounds' monoisotopic molecular masses (make sure that they are neutral masses)
- One with the compounds' names
- One with the compounds' chemical formulas

For the sake of simplicity, the lists of compound IDs, names and formulas will be placed in different columns. Just remember that they will be added by the same order, so the first ID in the IDs column corresponds to the first name in the names column and the first formula in the formulas column.

**See metanalysis_standard.py for functions about annotation**

In [ ]:
dbs = { # How you want the database to be called in the data: 'HMDB', ' PCY', 'DBK'
    'HMDB': {'File': 'hmdb_complete.xlsx', # The name of each file
             'Index_name': 'accession', # The name of the index in each database
             'Name_col': 'name', # The name column in each database
             'Mass_col': 'monisotopic_molecular_weight', # The mass column in each database. Can be None
             'Formula_col': 'chemical_formula'}, # The formula column in each database

    'LTS': {'File': 'LOTUS_DB_Ver2.xlsx', 
             'Index_name': 'Wikidata_ID',
             'Name_col': 'Name',
             'Mass_col': 'Mass',
             'Formula_col': 'Formula'},

    'DBK': {'File': 'Drugbank_small_molecules_labeled.csv', 
             'Index_name': 'accession',
             'Name_col': 'Name',
             'Mass_col': None,
             'Formula_col': 'Formula'}
}

In [ ]:
# testing it
for d in dbs:
    print(d, ' -> ', dbs[d]['File'], '|', dbs[d]['Index_name'],'|', dbs[d]['Mass_col'],'|', 
          dbs[d]['Name_col'],'|', dbs[d]['Formula_col'])

In [ ]:
# Upload databases
for d in dbs:
    print('Processing '+d)
    if dbs[d]['File'].endswith('.csv'):
        db = pd.read_csv(dbs[d]['File']).set_index(dbs[d]['Index_name'])
        if d == 'HMDB':
            db['name'] = db['name'].str.replace("b'", "")
            db['name'] = db['name'].str.replace("'", "")
    elif dbs[d]['File'].endswith('.xlsx'):
        db = pd.read_excel(dbs[d]['File']).set_index(dbs[d]['Index_name'])
        if d == 'HMDB':
            db['name'] = db['name'].str.replace("b'", "")
            db['name'] = db['name'].str.replace("'", "")
    else:
        raise ValueError('File Format not accepted. Only csv and xlsx files are accepted.')
    ##
    if dbs[d]['Mass_col'] == None:
        db['Calculated Mass'] = db[dbs[d]['Formula_col']].dropna().apply(metsta.calculate_monoisotopic_mass)
        dbs[d]['Mass_col'] = 'Calculated Mass'
    ##
    dbs[d]['DB'] = db
    print(d,'->', len(db.index), 'compounds')

#### Select which adducts you want to search for in the analysis

- Select adduct name and adduct mass shift in the dictionary - **At least one has to be selected to perform data annotation.**

This will calculate a mass based on the shift provided for each compound in the selected databases, which will then be used to search for and match to the _m/z_ or Neutral masses in your dataset.

Below, there are some examples for each of the 3 cases.

In [ ]:
electron_mass = 0.000548579909065

if idx_masses == 'Neutral':
    adducts_to_consider = {
        'Neutral': 0}
elif idx_masses == 'Positive':
    adducts_to_consider = {
        # Some common positive adducts to consider
        'H+': metsta.chemdict['H'] - electron_mass,
        'Na+': metsta.chemdict['Na'] - electron_mass,
        'K+': metsta.chemdict['K'] - electron_mass,
    }
elif idx_masses == 'Negative':
    adducts_to_consider = {
        # Some common negative adducts to consider - Confirm if these are correct
        'H-': - metsta.chemdict['H'] + electron_mass,
        'Cl-': metsta.chemdict['Cl'] + electron_mass,
    }
else:
    adducts_to_consider = {} # No annotation is possible

You can tune the parameter **ppm_margin** (first line of next cell) to select the maximum deviation you want for annotation.

We give an overview of the annotation here, but you should see only after filtration what remains.

In [ ]:
ppm_margin = 1
only_select_min_ppm = False # If True, when there are candidates for annotation with different ppm deviations, return only
# The ones with the lowest ppm deviation

# Annotation
annotated_data = file.copy()
    
metsta.metabolite_annotation(annotated_data, dbs, ppm_margin, mass_val_col,
                             adducts_to_consider=adducts_to_consider, only_select_min_ppm=only_select_min_ppm)

In [ ]:
annotated_data.loc[annotated_data['Matched HMDB IDs'].notnull()]

In [ ]:
meta_cols = [i for i in annotated_data.columns if i not in sample_cols]
meta_cols_ids = [i for i in meta_cols if 'IDs' in i]
meta_cols_names = [i for i in meta_cols if 'names' in i]
meta_cols_formulas = [i for i in meta_cols if 'formulas' in i]
meta_cols_mcounts = [i for i in meta_cols if 'match count' in i]
print(meta_cols)
print('------------')
print(meta_cols_ids)
print('------------')
print(meta_cols_names)
print('------------')
print(meta_cols_formulas)
print('------------')
print(meta_cols_mcounts)

# Step 1.3: De-duplicating annotations <a class="anchor" id="step-1_3"></a>

**Back to [Table of Contents](#toc)**

**This is by far the least streamlined section of the notebook. Function is specifically for MetaboScape data.**

Whether it is by MetaboScape or our script here in Python, due to the proximity of some m/z peaks, they can have the same exact compound annotation or in case of MetaboScape, formula.

The following part merges peaks that have the same compound annotation on MetaboScape or the different databases here into one single peak. This should happen since a compound should not be split into multiple peaks generally, there is no biological reason for being multiple peaks with palmitic acid, for example. Usually there is one peak that is the 'main' one with much higher intensities across the samples, although some cases this does not happen with the two _m/z_ peaks have or the annotation comes from two different adducts.

There is however a lot of problems and not ideal solutions being employed. In general, our procedure is the following.

1) See peaks that have the same metabolite annotation by other databases.

2) See if the other compound annotations do not have different annotations for those peaks.

3) If not, save the meta data of the compound and formula annotations by the different databases.

4) **Situation Trouble - If yes, then we may have a problem. If for example, HMDB puts two different compounds for the 2 _m/z_ peaks and LOTUS puts the same compound, it is fair to treat them as different peaks. HOWEVER, if there are more than two peaks assigned with the same formula, the following can happen. Let's imagine a scenario where HMDB puts the same compound for 4 _m/z_ peaks and LOTUS assigns to one of them one compound, to a second one a different compound and the last two ones does not assign a compound. What is the correct course of action? Right now, it just does not merge any of these peaks, but we could merge the two peaks that do not have an annotation by LOTUS. Would that be correct? Or should we merge with one of the two other peaks which have annotations by LOTUS. After all, they would normally be merged if not for the existence of two different LOTUS annotations. Hence, the problem.**

4) Then create the new peak, by keeping the highest intensity value in each sample from the different peaks (our intensity values come from the maximum value in the peak and not peak area.

5) Situation 1: If all the highest intensity values come from one _m/z_ peak, then that peak becomes the 'de facto' peak and all others are erased.

6) Situation 2: The highest intensity comes from at least two different _m/z_ peaks and ALL peaks come from the same adduct **(including ones that are not used for the merge)**. Then, the peak 'bucket label', 'Neutral Mass' or 'Probable _m/z_' columns become the weighted average (based on the average intensity of the peaks) of all the peaks with the same annotation. If there are no multiple adducts accepted for annotation, this is the situation used.

7) Situation 3: Identical to Situation 2 but there is at least one peak that comes from a different adduct based on Probable _m/z_ column. Then, the peak 'bucket label', 'Neutral Mass'/'Probable _m/z_' columns become identical to the peak which has the highest average intensity of all the peaks with the same annotation.

8) This process is repeated for MetaboScape annotations and all databases first. And is then used for Smart Formula. **Usually the number of de-duplications made by each database should decrease since when you de-duplciate duplicate assignments by one database, you are usually de-duplicating in others.**


**The problem mentioned should be rare. However, ALWAYS check the merge_problems variable. If it is NOT empty, then those merge problems issues might exist. We do not currently have an automatic answer for them. They should be seen on a case by case basis.**

In [ ]:
if 'Name' in meta_cols:
    #mcid = ['Name'] + meta_cols_ids
    mcid = ['Name'] + list(dbs.keys())
    
else:
    mcid = list(dbs.keys())
    
if 'Formula' in meta_cols:
    mcid = mcid + ['Formula']

if 'm/z' in meta_cols:
    mz_col = True

else:
    mz_col=False

Duplicate (or more) annotations report

In [ ]:
for col in mcid:
    n_duplicates = []
    if col not in ['Name', 'Formula']:
        col_alt = 'Matched '+col+' IDs'
    else:
        col_alt = col
        col = 'MetaboScape ' + col
    for i in annotated_data[annotated_data[col_alt].notnull()][col_alt]:
        a = 0
        for j in annotated_data[annotated_data[col_alt].notnull()][col_alt]:
            if i==j:
                if a == 1:
                    #print(i)
                    n_duplicates.append(i)
                    break
                a+=1
    print(col)
    print('Nº of same annotations on multiple peaks:        ', len(n_duplicates))
    print('Total number of annotations for these cases:     ', len(pd.Series(n_duplicates, dtype='object').value_counts()))
    if len(pd.Series(n_duplicates, dtype='object').value_counts()) == 0:
        print('Maximum number of peaks with the same annotation:', 0)
    else:
        print('Maximum number of peaks with the same annotation:', pd.Series(n_duplicates).value_counts().iloc[0])
    print('---------')

In [ ]:
old_data = annotated_data.copy()

Select if you want to perform de-duplication.

In [ ]:
perform_deduplication = True # False
verbose = False

In [ ]:
multiple_adds = True if len(adducts_to_consider) > 1 else False

if perform_deduplication:
    annotated_data,mergings_performed,merging_situations,merge_description,merge_problems = metsta.duplicate_disambiguator(
        annotated_data, # Our data
        sample_cols, # Columns where the samples are
        mcid=mcid,
        mass_col=mass_val_col,
        multiple_adds=multiple_adds, # If you have an m/z column
        verbose=verbose) # If you want a more detailed output while the function runs
else:
    mergings_performed, merging_situations, merge_description, merge_problems = [], [], [], []

### Seeing problems in merging

**Example for this specific dataset**

In this case, there are nine of them, although some are repeating. For example, the column with 0 and 5 are due to the same annotations as well as columns 1, 2, 6 and 7. So, in reality there are only 5 problems.

Furthermore, columns 3 and 4 are due to different SmartFormula annotations (less reliable than annotations generally), so we can merge them by forcing the Formula to be the one from the peak with the highest average intensity.

Thus we have 3 remaining problems. The problem is only really real when there are more than 2 different peaks, so in this case, only the problem with column 0 (same as with 5) should be the problem,. But, let's still see them to prove this.

Let's see them individually.

These potential problems should be incredibly rare.

In [ ]:
problem_df = pd.DataFrame(merge_problems)
problem_df

In [ ]:
# Force merging the cases where the problem is the Formula
# The Smart Formula that remains is the one corresponding to the  peak with the highest average intensity
k_to_remove = []
if len(problem_df)>0: 
    for k, v in merge_problems.items():
        if v['Poss. Reason'] == 'Formula':
            problem_case = v
            if problem_case['Col Id.'] not in ['Name', 'Formula']:
                db_problem = 'Matched '+problem_case['Col Id.']+' IDs'
            else:
                db_problem = problem_case['Col Id.']
            idx_to_merge = list(k)

            if len(idx_to_merge)>1:
                annotated_data, desc = metsta.individually_merging(
                    annotated_data, # Data
                    idx_to_merge, # Your idx to merge
                    sample_cols, mass_val_col, mcid, multiple_adds=multiple_adds)

                desc[list(desc.keys())[0]]['Situation'] = desc[list(desc.keys())[0]]['Situation'] + '- Formula Problem'
                # See if the description of the merging is what you wanted to achieve

                # Supplementing the information to merging descriptors
                merge_description[list(desc.keys())[0]] = desc[list(desc.keys())[0]]
                if desc[list(desc.keys())[0]]['Situation'] in merging_situations:
                    merging_situations[desc[list(desc.keys())[0]]['Situation']] += 1
                else:
                    merging_situations[desc[list(desc.keys())[0]]['Situation']] = 1
                mergings_performed[desc[list(desc.keys())[0]]['DB']] += 1
                k_to_remove.append(k)

for k in k_to_remove:
    merge_problems.pop(k)

In [ ]:
if len(merging_situations)>0:
    merging_situations['Problems'] = len(merge_problems)
problem_df = pd.DataFrame(merge_problems)
problem_df

1) L-Cystathionine (columns 1, 2, 6 and 7)

You can see L-Cystathionine formula is C24H48NO7P that is the annotation that leads to problems in column 6 and 7.

The problem comes from an extra annotation in HMDB and LTS in one of the peaks: 'Acetamiprid' (in both cases). Different annotations so no issue.

In [ ]:
annotated_data[annotated_data['Name'] == 'PC_16:1_9Z_0:0_'][meta_cols]

In [ ]:
ex = pd.DataFrame()
if filename == '5yeasts_notnorm.csv':
    ex = annotated_data[annotated_data['Name'] == 'L-Cystathionine'][meta_cols]
ex

In [ ]:
ex = pd.DataFrame()
if filename == '5yeasts_notnorm.csv':
    ex = annotated_data[annotated_data['Name'] == 'L-Cystathionine']['Matched HMDB names'].values
ex

**2) PC_16:1_9Z_0:0_ (C26H52NO7P) - The major problem**

Here, we have a representation of the major possible problem which hopefully you will not have to deal with.

Solving the issue:

- See the problem
- See the idxs you want to manually merge (here we merge every peak other than the one with the extra LOTUS annotations since the first peak by HMDB and LOTUS has the same formula as all other peaks that do not have annotations.
- See the DB where this problem happenned ('Name' in this case) and the annotation('PC_16:1_9Z_0:0_' in this case).
- Use the `individually_merging` function and add the result to previous.

In [ ]:
if filename == '5yeasts_notnorm.csv':
    problem_case = merge_problems[('493.3168156897 Da',
  '493.3142206664 Da',
  '493.3192028044 Da',
  '493.3133493312 Da',
  '493.3202367046 Da',
  '493.3215300481 Da')]

In [ ]:
idx_to_merge = []
if filename == '5yeasts_notnorm.csv':
    if problem_case['Col Id.'] not in ['Name', 'Formula']:
        db_problem = 'Matched '+problem_case['Col Id.']+' IDs'
    else:
        db_problem = problem_case['Col Id.']

    # Grab all the idxs of the annotation
    idx_to_merge = list(('493.3168156897 Da','493.3142206664 Da','493.3192028044 Da','493.3133493312 Da',
                          '493.3202367046 Da','493.3215300481 Da'))

    idx_to_merge.remove('493.3192028044 Da') # Remove the one we don't want to merge in this case, the one with the extra
    # LOTUS annotation

idx_to_merge # Check if it is correct

In [ ]:
desc = []
if filename == '5yeasts_notnorm.csv':
    annotated_data, desc = metsta.individually_merging(
        annotated_data, # Data
        idx_to_merge, # Your idx to merge
        sample_cols, mass_val_col, mcid, multiple_adds=True)

# See if the description here is what you wanted to achieve
desc

In [ ]:
if filename == '5yeasts_notnorm.csv':
    # Supplementing the information to merging descriptors
    merge_description[list(desc.keys())[0]] = desc[list(desc.keys())[0]]
    if desc[list(desc.keys())[0]]['Situation'] in merging_situations:
        merging_situations[desc[list(desc.keys())[0]]['Situation']] += 1
    else:
        merging_situations[desc[list(desc.keys())[0]]['Situation']] = 1
    mergings_performed[desc[list(desc.keys())[0]]['DB']] += 1

3) C24H48NO7P

Same Formula but different MetaboScape annotations, thus no major problem.

In [ ]:
ex = pd.DataFrame()
if filename == '5yeasts_notnorm.csv':
    ex = annotated_data[annotated_data['Formula'] == 'C26H52NO7P'][meta_cols]
ex

#### Example of merging

See how there were 3 peaks with PE_16:1_9Z_0:0_ annotation with MetaboScape with one of them having an HMDB and Drugbank annotation in our software and another one with a LOTUS annotation. The method used was overwrite keeping the first of the three peaks as the 'true peak' and we moved the LOTUS annotation to that peak.

In [ ]:
ex = pd.DataFrame()
if filename == '5yeasts_notnorm.csv':
    ex = old_data.loc[[i for i in old_data.index if old_data.loc[i,'Name'] == 'PE_16:1_9Z_0:0_']][meta_cols]
ex

In [ ]:
ex = pd.DataFrame()
if filename == '5yeasts_notnorm.csv':
    ex = annotated_data.loc[[
        i for i in annotated_data.index if annotated_data.loc[i,'Name'] == 'PE_16:1_9Z_0:0_']][meta_cols]
ex

In [ ]:
ex = pd.DataFrame()
if filename == '5yeasts_notnorm.csv':
    ex = old_data.loc[[i for i in old_data.index if old_data.loc[i,'Name'] == 'PE_16:1_9Z_0:0_']][sample_cols]
ex

### Description of merging process

In [ ]:
m_desc = pd.DataFrame(merge_description)
m_desc

In [ ]:
if len(m_desc)>0:
    print('Nº of Mergings:            ', len(m_desc.columns))
    print('Nº of Peaks merged:        ', m_desc.loc['Nº merged peaks'].sum())
    print('Nº of Peaks dropped:       ', m_desc.loc['Nº merged peaks'].sum() - len(m_desc.columns))
    print('Nº of Peaks after merging: ', len(annotated_data.index))

In [ ]:
mergings_performed

In [ ]:
merging_situations

In [ ]:
print('Checking all matches')
if len(dbs) == 0:
    if 'Name' in annotated_data.columns:
        for i in tqdm(annotated_data.index):
            df = annotated_data.loc[[i]]
            hasmatch = df['Name'].notnull().values.any()
            annotated_data.at[i, 'Has Match?'] = hasmatch
        print('Nº of Annotated Compounds:', annotated_data[annotated_data['Has Match?'] == True].sum())
    else:
        print('No Check Neccessary')
else:
    if 'Name' in annotated_data.columns:
        cols_to_see = ['Name'] + list(annotated_data.columns[-len(dbs)*5:])
    else:
        cols_to_see = list(annotated_data.columns[-len(dbs)*5:])
    annotated_data['Has Match?'] = np.nan
    for i in tqdm(annotated_data.index):
        df = annotated_data.loc[[i]]
        hasmatch = df[cols_to_see].notnull().values.any()
        annotated_data.at[i, 'Has Match?'] = hasmatch
    print('Nº of Annotated Compounds:', (annotated_data['Has Match?'] == True).sum())
    print('---------------')

annotated_data.info(verbose= True)
annotated_data   

# Step 2: Basic processing and pre-treatment <a class="anchor" id="step-2"></a>

**Back to [Table of Contents](#toc)**

**Functions in metanalysis_standard!**

These functions are compilations from the pre-treatments available in the **Metabolinks** Python package.

Each step of this process has a different associated function that explain different methods available to do those steps that are included in the big all-including `filtering_pretreatment` function. By our experience, the default option in the different functions are the most common ones to use. There are way more options to use for all these steps, for example, for Missing Value Imputation that can be applied that are not present here and would have to be implemented.

This returns four DataFrames:
- **treated_data** - Data after filtering and pre-treatment with the samples ready for statistical analysis.
- **processed_data** - Data after filtering and only normalization with samples and meta data used for compound finding and distinguishing between common and exclusive metabolites.
- **univariate_data** - Data after filtering, imputation and only normalization used for fold change calculation in univariate analysis.
- **meta_data** - Meta data with compound annotation and formulas for later.
- **bin_data** - treated_data but with BinSim just because.

**The procedures to be used need to be chosen by the user.**

### Feature Filtering - `basic_feat_filtering` function

This part removes the features that appear only in one sample (likely experimental artifacts and not real metabolites). If this is already done, skip this part by turning **_filt_method argument_ in `filtering_pretreatment` function to None**.

**Available methods**: 'total_samples' (_default_), 'class_samples', None.

**There can also be an extra step just keeping masses with annotations**: 'Formula', 'Name', None (_default_). Explained in the function.

### Data Pre-Treatment

There are many different ways these can be used but in general there are four categories: 'Missing Value Imputation', 'Normalization', 'Transformations' and 'Scaling' each with their options. If you do not want some types of pre-treatment, select None for that specific category (except missing value imputation, that HAS to be done).

#### Note: If data was already normalized in MetaboScape, skip normalization by making _norm_ argument in `filtering_pretreatment` function to None and remember to remove Leucine Enkephalin peak (reference feature) if you have it (see cell imediately above step 1.2).

Each different method for each category is explained in their respective functions. Each category has also a keyword (kw) that can be added since many methods have one parameter that can be changed. That keyword becomes that parameter.

**Missing Value Imputation** (`missing_value_imputer`): 'min_sample' (_Default_), 'min_feat', 'min_data', 'zero'.

**Normalization** by (`normalizer`): 'ref_feat' (_Default_), 'total_sum', 'PQN', 'Quantile', None.

**Transformation** (`transformer`): 'glog' (_Default_), None.

**Scaling** (`scaler`): 'pareto' (_Default_), 'mean_center', 'auto', 'range', 'vast', 'level', None.

Furthermore, **Binary Simplification** (BinSim) is also returned as well and kept in bin_data.

#### Choose your feature filtering and data pre-treatment as you see fit 

##### See options for each step above and more detail in the function itself in the metanalysis_standard.py file.

In [ ]:
# Filtering based on number of times features appear
filt_method='total_samples' # 'total_samples', 'class_samples', None
filt_kw=2 # Nº of minimum samples of the dataset ('total_samples') or class ('class_samples') features have to appear in
extra_filt=None # Filtering based on annotation of features 'Formula', 'Name' or None

# Missing Value Imputations
mvi='min_sample' # 'min_sample' (Default), 'min_feat', 'min_data', 'zero'
mvi_kw=1/5 # Specific Keyword for MVI method

# Normalization
norm='total_sum' # 'ref_feat' (Default), 'total_sum', 'PQN', 'Quantile', None
norm_kw='555.2692975341 Da' # Specific keyword for Normalization method

# Transformation
tf='glog' # 'glog' (Default), None
tf_kw=None # Specific keyword for Transformation

# Scaling
scaling='pareto' # 'pareto' (Default), 'mean_center', 'auto', 'range', 'vast', 'level', None
scaling_kw=None # Specific keyword for Scaling

# Change the parameters in the variables above
treated_data, processed_data, univariate_data, meta_data, bin_data = metsta.filtering_pretreatment(
                  annotated_data, target,sample_cols,
                  filt_method, filt_kw, extra_filt, # Filtering 
                  mvi, mvi_kw, # Missing value imputation
                  norm, norm_kw, # Normalization
                  tf, tf_kw, # Transformation
                  scaling, scaling_kw) # Scaling

In [ ]:
treated_data

In [ ]:
#processed_data.info()
processed_data

# Step 3: Find Common and Exclusive metabolites between the groups <a class="anchor" id="step-3"></a>

**Back to [Table of Contents](#toc)**

First, make a dataframe for each class with only the features that appear in those classes. We will also make another set of DataFrames only considering annotated metabolites.

In [ ]:
# Sample specific data frames
groups = {}
group_dfs = {}

group_dfs_ids = {}
#id_selection = "Name"

for cl in classes:
    groups[cl] = []
    
for c, t in zip(processed_data[sample_cols].columns, target):
    for g in groups:
        if g == t:
            groups[g].append(c)
            
for g in groups:
    for c, t in zip(processed_data[sample_cols].columns, target):
        if g == t:
            group_dfs[g] = processed_data.dropna(subset= groups[g], thresh=1)
            group_dfs_ids[g] = group_dfs[g].iloc[[
                i for i in range(len(group_dfs[g]['Has Match?'])) if group_dfs[g]['Has Match?'].iloc[i]]]

for df in group_dfs:
    print(df,  '------>', len(group_dfs[df]), 'metabolites from which', len(group_dfs_ids[df]), f'have matches')

Now you have a specific dataframe for each individual group in your samples. They are all in a dictionary called group_dfs with the format name : dataframe.

In [ ]:
# For example....
print(f"Here's the dataframe for one of the classes: {list(group_dfs.keys())[0]}")
group_dfs[list(group_dfs.keys())[0]]

We can now see the common and exclusive metabolites between these DataFrames.

**See corresponding functions in metanalysis_standard.**

In [ ]:
# Common to all
common_all = metsta.common(group_dfs.values())
common_all_id = metsta.common(group_dfs_ids.values())
print(len(common_all.index), f'metabolites are common to all group, {len(common_all_id.index)} with matches.')

print('    ')

# Common to two or more - Might become unintelligible if you have too many classes
for n in range(2, len(group_dfs)+1):
    for comb in itertools.combinations(group_dfs, n):
        df_list = []
        labels_list = []
        df_id_list = []
        for c in comb:
            labels_list.append(c)
            df = group_dfs[c]
            df_list.append(df)
            df_id_list.append(group_dfs_ids[c])
        df_common = metsta.common(df_list)
        df_id_common = metsta.common(df_id_list)
        for s in group_dfs:
            if s not in labels_list:
                exclude = group_dfs[s]
                df_common = df_common.loc[~(df_common.index.isin(exclude.index))]
                df_id_common = df_id_common.loc[~(df_id_common.index.isin(group_dfs_ids[s].index))]
        print(len(df_common.index), f'metabolites ({len(df_id_common.index)} with matches) are common to {comb}.') 
        
print('     ')

# Exclusive to only one group
exclusives = metsta.exclusive(group_dfs.values())
exclusives_id = metsta.exclusive(group_dfs_ids.values())
exc_id = dict(zip(group_dfs, exclusives_id))
exc = dict(zip(group_dfs, exclusives))
for g in group_dfs:
    print(len(exc[g].index), f'metabolites are exclusive to {g}, {len(exc_id[g].index)} with matches.')

**Venn Diagram**

Plots made using the pyvenn git-hub repository (https://github.com/tctianchi/pyvenn).

In [ ]:
# Make a Venn diagram
labels = venn.get_labels([group_dfs[i].index for i in group_dfs], fill=['number'])
labels_ids = venn.get_labels([group_dfs_ids[i].index for i in group_dfs_ids], fill=['number'])

labels_all = {}
for i, j in labels.items():
    labels_all[i] = j + f' ({labels_ids[i]})'
    
c = [(c[0], c[1], c[2], 0.3) for c in colours]

if len(classes) == 2:
    fig, ax = venn.venn2(labels_all, names=classes, figsize=(8,8), fontsize=11, colors=c, constrained_layout=True) # 2 Classes
    plt.text(0.5,0, 'Nº of peaks (Nº of matched compounds)', fontsize=12, horizontalalignment='center')
elif len(classes) == 3:
    fig, ax = venn.venn3(labels_all, names=classes, figsize=(8,8), fontsize=11, colors=c, constrained_layout=True) # 3 Classes
    plt.text(0.5,-0.05, 'Nº of peaks (Nº of matched compounds)', fontsize=12, horizontalalignment='center')
elif len(classes) == 4:
    fig, ax = venn.venn4(labels_all, names=classes, figsize=(8,8), fontsize=11, colors=c, constrained_layout=True) # 4 Classes
    plt.text(0.5,0.05, 'Nº of peaks (Nº of matched compounds)', fontsize=12, horizontalalignment='center')
elif len(classes) == 5:
    fig, ax = venn.venn5(labels_all, names=classes, figsize=(8,8), fontsize=11, colors=c, constrained_layout=True) # 5 Classes
    plt.text(0.5,0, 'Nº of peaks (Nº of matched compounds)', fontsize=12, horizontalalignment='center')
elif len(classes) == 6:
    fig, ax = venn.venn6(labels_all, names=classes, figsize=(8,8), fontsize=11, colors=c, constrained_layout=True) # 6 Classes
    plt.text(0.5,0.2, 'Nº of peaks (Nº of matched compounds)', fontsize=12, horizontalalignment='center') 
else:
    print(f'Venn Diagram can currently only be made with 2 to 6 different classes. You currently have {len(classes)} classes.')

# Save the Venn Diagram
#if len(classes) < 7:
#    fig.savefig('VennDiagram_plot.png', dpi=400)

**UpSetPlot** (not recommended for more than 6 classes also)

UpSetPlots made using the package UpSetPlot (https://pypi.org/project/UpSetPlot/0.8.0/)

See parameters two cells down

In [ ]:
# Make an upsetplot
groups_dict = {}
for df in group_dfs:
    groups_dict[df] = group_dfs[df].index
ups = from_contents(groups_dict)

In [ ]:
# Parameters
counts = True # Show absolute counts of metabolites on top of bars
percentages = False # Show percentage of metabolites on top of bars
include_annotated = False # Include annotated compounds as a bar of a different colour
annotated_colour = 'Red' # Choose the color for the bar
annotated_counts = False # Show absolute counts of annotated metabolites on top of the annotated metabolites bars

In [ ]:
# Plotting UpSetPlot
f,ax = plt.subplots(1,1, constrained_layout=True)
ax.axis('Off')

# Plot Main Upset Plot
ax_dict = upsetplot.plot(ups, f, subset_size='count', show_counts=counts, show_percentages=percentages,
                         sort_categories_by='input', include_empty_subsets=include_annotated)

# Put counts of only annotated features if include_annotated = True
if include_annotated:
    groups_dict_id = {}
    for df in group_dfs_ids:
        groups_dict_id[df] = group_dfs_ids[df].index
    ups_id = from_contents(groups_dict_id)
    UpSet(ups_id, subset_size='count',facecolor=annotated_colour, sort_categories_by='input',
          include_empty_subsets=include_annotated).plot_intersections(ax_dict['intersections'])
    a=0
    # Put the counts over the red, you might have to adjust the +15 part to something else depending on the counts you have
    if annotated_counts:
        for i in upsetplot.query(ups_id,sort_categories_by='input', include_empty_subsets=True).subset_sizes:
            #print()
            ax_dict['intersections'].text(
                a,i+15, i, color='red', fontsize=10, zorder=15, horizontalalignment='center', weight="bold")
            a +=1

# Save the UpSet Plot
#fig.savefig('UpSetPlot_plot.png', dpi=400)

#### Setting an Excel file with the common and exclusive (to each class) annotated compounds

Change **GENERATE_Excel_file** to True if you want to generate the file.

In [ ]:
# Setting the excel files for exclusive compounds
exclusive_dfs = {}
for i in exc_id.keys():
    df_temp = pd.DataFrame(index=exc_id[i].index)
    df_temp['Appear in Class Samples'] = exc_id[i].loc[:, sample_cols].notnull().sum(axis=1)
    df_temp['% of Class Samples'] = (exc_id[i].loc[:, sample_cols].notnull().sum(axis=1)) / target.count(i) * 100

    if 'Name' in exc_id[i].columns:
        df_temp['MetaboScape Match'] = exc_id[i]['Name']
    if 'Formula' in exc_id[i].columns:
        df_temp['MetaboScape Formula'] = exc_id[i]['Formula']
    for col in meta_cols:
        if col not in ['Neutral Mass', 'Probable m/z', 'm/z', 'Name', 'Formula', 'Has Match?']:
            df_temp[col] = exc_id[i][col]
    exclusive_dfs[i] = df_temp

In [ ]:
# Example of Exclusive df, ordered by the number of samples of that class they appear in
print('Example for:', classes[0])
exclusive_dfs[classes[0]].sort_values(by='Appear in Class Samples', ascending=False)

In [ ]:
# Setting the excel files for common compounds
common_temp = pd.DataFrame(index=common_all_id.index)
common_temp['Appear in Samples'] = common_all_id.loc[:, sample_cols].notnull().sum(axis=1)
common_temp['% of Samples'] = (common_all_id.loc[:, sample_cols].notnull().sum(axis=1)) / len(target) * 100
if 'Name' in common_all_id.columns:
    common_temp['MetaboScape Match'] = common_all_id['Name']
if 'Formula' in common_all_id.columns:
    common_temp['MetaboScape Formula'] = common_all_id['Formula']
for col in meta_cols:
    if col not in ['Neutral Mass', 'Probable m/z', 'm/z', 'Name', 'Formula', 'Has Match?']:
        common_temp[col] = common_all_id[col]
common_temp

In [ ]:
GENERATE_Excel_file = False # Change to True if you want to generate the file
if GENERATE_Excel_file:
    writer = pd.ExcelWriter('Common_Exclusive_Compounds.xlsx', engine='xlsxwriter')

    common_temp.to_excel(writer, sheet_name='Common')

    text_format = writer.book.add_format({'text_wrap' : True, 'valign': 'top'})
    for i in range(1, len(common_temp.columns)+1):
        width=18
        if i in [1,2]:
            width=8
        elif common_temp.columns[i-1].endswith('IDs'):
            width=15
        elif common_temp.columns[i-1].endswith('count'):
            width=8
        elif common_temp.columns[i-1].endswith('names') or common_temp.columns[i-1].endswith('Name'):
            width=40
        writer.sheets['Common'].set_column(i,i,width,text_format)

    header_format = writer.book.add_format({'bold': True, 'text_wrap': True, 'valign': 'top'})
    # Overwrite both the value and the format of each header cell
    for col_num, value in enumerate(common_temp.columns.values):
        writer.sheets['Common'].write(0, col_num + 1, value, header_format)

    for a in exclusive_dfs.keys():
        exclusive_dfs[a].to_excel(writer, sheet_name=str(a)+' Exclusive')

        text_format = writer.book.add_format({'text_wrap' : True, 'valign': 'top'})
        for i in range(1, len(exclusive_dfs[a].columns)+1):
            width=18
            if i in [1,2]:
                width=8
            elif exclusive_dfs[a].columns[i-1].endswith('IDs'):
                width=15
            elif exclusive_dfs[a].columns[i-1].endswith('count'):
                width=8
            elif exclusive_dfs[a].columns[i-1].endswith('names') or exclusive_dfs[a].columns[i-1].endswith('Name'):
                width=40
            writer.sheets[str(a)+' Exclusive'].set_column(i,i,width,text_format)

        header_format = writer.book.add_format({'bold': True, 'text_wrap': True, 'valign': 'top'})
        # Overwrite both the value and the format of each header cell
        for col_num, value in enumerate(exclusive_dfs[a].columns.values):
            writer.sheets[str(a)+' Exclusive'].write(0, col_num + 1, value, header_format)

    writer.close()

# Step 4: Unsupervised Statistical Analysis <a class="anchor" id="step-4"></a>

**Back to [Table of Contents](#toc)**

Unsupervised analysis means that the algorithms here do not receive the information of the different class labels.

Here, we show PCA and Hierarchical Clustering (HCA) Analysis.

The following functions were taken from the code git-hub repository 'binsim_paper' (from the files 'paper_binsim_data_prep.ipynb' and 'paper_binsim_unsupervised.ipynb').

In [ ]:
def plot_PCA(principaldf, label_colors, components=(1,2), title="PCA", ax=None):
    "Plot the projection of samples in the 2 main components of a PCA model."
    
    if ax is None:
        ax = plt.gca()
    
    loc_c1, loc_c2 = [c - 1 for c in components]
    col_c1_name, col_c2_name = principaldf.columns[[loc_c1, loc_c2]]
    
    #ax.axis('equal')
    ax.set_xlabel(f'{col_c1_name}')
    ax.set_ylabel(f'{col_c2_name}')

    unique_labels = principaldf['Label'].unique()

    for lbl in unique_labels:
        subset = principaldf[principaldf['Label']==lbl]
        ax.scatter(subset[col_c1_name],
                   subset[col_c2_name],
                   s=50, color=label_colors[lbl], label=lbl)

    #ax.legend(framealpha=1)
    ax.set_title(title, fontsize=15)

def plot_ellipses_PCA(principaldf, label_colors, components=(1,2),ax=None, q=None, nstd=2):
    "Plot confidence ellipses of a class' samples based on their projection in the 2 main components of a PCA model."
    
    if ax is None:
        ax = plt.gca()
    
    loc_c1, loc_c2 = [c - 1 for c in components]
    points = principaldf.iloc[:, [loc_c1, loc_c2]]
    
    #ax.axis('equal')

    unique_labels = principaldf['Label'].unique()

    for lbl in unique_labels:
        subset_points = points[principaldf['Label']==lbl]
        plot_confidence_ellipse(subset_points, q, nstd, ax=ax, ec=label_colors[lbl], fc='none')

def color_list_to_matrix_and_cmap(colors, ind, axis=0):
        if any(issubclass(type(x), list) for x in colors):
            all_colors = set(itertools.chain(*colors))
            n = len(colors)
            m = len(colors[0])
        else:
            all_colors = set(colors)
            n = 1
            m = len(colors)
            colors = [colors]
        color_to_value = dict((col, i) for i, col in enumerate(all_colors))

        matrix = np.array([color_to_value[c]
                           for color in colors for c in color])

        matrix = matrix.reshape((n, m))
        matrix = matrix[:, ind]
        if axis == 0:
            # row-side:
            matrix = matrix.T

        cmap = mpl.colors.ListedColormap(all_colors)
        return matrix, cmap

def plot_dendogram(Z, leaf_names, label_colors, title='', ax=None, no_labels=False, labelsize=12, **kwargs):
    if ax is None:
        ax = plt.gca()
    hier.dendrogram(Z, labels=leaf_names, leaf_font_size=10, above_threshold_color='0.2', orientation='left',
                    ax=ax, **kwargs)
    #Coloring labels
    #ax.set_ylabel('Distance (AU)')
    ax.set_xlabel('Distance (AU)')
    ax.set_title(title, fontsize = 15)
    
    #ax.tick_params(axis='x', which='major', pad=12)
    ax.tick_params(axis='y', which='major', labelsize=labelsize, pad=12)
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    #xlbls = ax.get_xmajorticklabels()
    xlbls = ax.get_ymajorticklabels()
    rectimage = []
    for lbl in xlbls:
        lbl_text = lbl.get_text()
        if type(list(label_colors)[0]) == np.float64:
            lbl_text = float(lbl_text)
        col = label_colors[lbl_text]
        lbl.set_color(col)
        #lbl.set_fontweight('bold')
        if no_labels:
            lbl.set_color('w')
        rectimage.append(col)

    cols, cmap = color_list_to_matrix_and_cmap(rectimage, range(len(rectimage)), axis=0)

    axins = inset_axes(ax, width="5%", height="100%",
                   bbox_to_anchor=(1, 0, 1, 1),
                   bbox_transform=ax.transAxes, loc=3, borderpad=0)

    axins.pcolor(cols, cmap=cmap, edgecolors='w', linewidths=1)
    axins.axis('off')

### Principal Component Analysis (PCA)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(6,6)) # Change the size of the figure

principaldf, var, loadings = metsta.compute_df_with_PCs_VE_loadings(treated_data, 
                                       n_components=2, # Select number of components to calculate
                                       whiten=True, labels=target, return_var_ratios_and_loadings=True)

# Plot PCA
ax.axis('equal')
lcolors = label_colours

plot_PCA(principaldf, lcolors, 
         components=(1,2), # Select components to see
         title='', # Select title of plot
         ax=ax)

# Remove ellipses by putting a # before the next line
plot_ellipses_PCA(principaldf, 
                  lcolors, 
                  components=(1,2), # Select components to see
                  ax=ax, 
                  q=0.95) # Confidence ellipse with 95% (q) confidence

ax.set_xlabel(f'PC 1 ({var[0] * 100:.1f} %)', size=15) # Set the size of labels
ax.set_ylabel(f'PC 2 ({var[1] * 100:.1f} %)', size=15) # Set the size of labels

plt.legend(fontsize=15) # Set the size of labels
plt.grid() # If you want a grid or not
plt.show()
#f.savefig('Name_PCAplot.png', dpi=400) # Save the figure

### Hierarchical Clustering Analysis (HCA)

Performing Hierarchical Clustering.

Distance metrics: 'euclidean' is the default, others are in https://docs.scipy.org/doc/scipy/reference/spatial.distance.html.

Linkage metrics: **'ward', 'average'**, 'centroid', 'single', 'complete', 'weighted', 'median'.

In [ ]:
metric = 'euclidean' # Select distance metric
method = 'ward' # Select linkage method

distances = dist.pdist(treated_data, metric=metric)
Z = hier.linkage(distances, method=method)

hca_res = {'Z': Z, 'distances': distances}

In [ ]:
# Plot HCA
with sns.axes_style("white"):
    f, ax = plt.subplots(1, 1, figsize=(4, 4), constrained_layout=True) # Set Figure Size
    plot_dendogram(hca_res['Z'], 
                   target, ax=ax,
                   label_colors=label_colours,
                   title='', # Select title
                   color_threshold=0) # Select a distance threshold from where different sets of lines are coloured

    plt.show()
    #f.savefig('Name_HCAplot.png', dpi=400) # Save the figure

If you want a version of a dendrogram more easy to change parameters:

In [ ]:
fig = plt.figure(figsize=(12,4))
# Plotting the dendrogram, see https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html
# For details on how you can change different aspects of the dendrograms
dn = hier.dendrogram(hca_res['Z'], labels=target,
                     leaf_font_size=13,
                     above_threshold_color='b')
# Coloring labels
ax = plt.gca()
ax.set_ylabel('Distance (UA)')
# Coloring the labels with their specific colours
xlbls = ax.get_xmajorticklabels()
for lbl in xlbls:
    lbl_text = lbl.get_text()
    if type(list(label_colours)[0]) == np.float64:
        lbl_text = float(lbl_text)
    lbl.set_color(label_colours[lbl_text])

# Step 5: Supervised Statistical Analysis <a class="anchor" id="step-5"></a>

**Back to [Table of Contents](#toc)**

Supervised analysis means that the algorithms have access to label information. This means they are **not** indicated for the purpose of seeing if there are differences between classes/samples, only for seeing which metabolites are most important for those differences.

The supervised statistical analysis methods currently implemented in this notebook are:
- Random Forest Models (RFs)
- Partial Least Squares (PLS)
- Extreme Gradient Boosting (XGBoost)

They all support both regression and classification problems, but may not be equally suitable for all use cases.

XGBoost has thus far performed poorly in Binary classification problems, and both XGBoost and Random Forests may take a long time to run for regression problems, depending on the hyperparameters chosen.

**Functions for this step are in metanalysis_standard.py and are an adaptation of functions in multianalysis.py (from the BinSim paper).**

First, you must intend if you intend to use the methods below to perform regressions or classifications. You may also change this in the parameter of individual method functions.

If you pick regressions, please maake sure that the "class" labels are numerical values.

In [ ]:
regression = False

## Step 5.1: Random Forest <a class="anchor" id="step-5_1"></a>

**Back to [Table of Contents](#toc)**

First: Minor optimization of the number of trees (200 is a good number to use though) - see when the accuracy of the model stops increasing and starts fluctuating around a certain value (that should be the minimum number of trees to use).

In [ ]:
# Select a random seed (number between the ()) if you don't want the results to change every time you run the code
np.random.seed()

# See maximum number of trees to search
top_tree_in_grid=300

# Vector with values for the parameter n_estimators
# Models will be built from 10 to 300 trees in 5 tree intervals
values = {'n_estimators': range(10,top_tree_in_grid,5)}

if regression:
    rf = skensemble.RandomForestRegressor(n_estimators=200)
else:
    rf = skensemble.RandomForestClassifier(n_estimators=200)
    
clf = GridSearchCV(rf, values, cv=3, n_jobs=-1) # Change cv to change cross-validation

print('Fitting RFs...', end=' ')

RF_optim = {'Treated':{}, 'BinSim':{}}
clf.fit(treated_data, target) # Fitting the data to RF models with all the different number of trees

# Storing results
RF_optim['Treated']['scores'] = list(clf.cv_results_['mean_test_score'])
RF_optim['Treated']['n_trees'] = list(clf.cv_results_['param_n_estimators'])

# BinSim turn
clf.fit(bin_data, target) # Fitting the data to RF models with all the different number of trees

# Storing results
RF_optim['BinSim']['scores'] = list(clf.cv_results_['mean_test_score'])
RF_optim['BinSim']['n_trees'] = list(clf.cv_results_['param_n_estimators'])


print('Done!')

In [ ]:
# Plotting the results and adjusting parameters of the plot
with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1.2):
        f, ax = plt.subplots(1, 1, figsize=(6,6), constrained_layout=True) # Set Figure Size

        c_map = sns.color_palette('tab10', 10)

        for treatment, c in zip(RF_optim.keys(), c_map):
            ax.plot(RF_optim[treatment]['n_trees'], [s*100 for s in RF_optim[treatment]['scores']], label=treatment, color=c)
        
        ax.set_ylabel('Random Forest CV Mean Accuracy (%)', fontsize=15) # Set the y_label and size
        ax.set_title('RF Optimization', fontsize=18) # Set the title and size
        ax.set_ylim([30,101]) # Set the limits on the y axis

        #f.suptitle('Optimization of the number of trees')
        ax.legend(fontsize=15) # Set the legend and size
        plt.show()

### Fitting the RF model

**See details of `RF_model` function (model fitting AND evaluation) in metanalysis_standard.py. Credit to initial function to the BinSim paper.**

In [ ]:
def RF_model(df, y, regres, return_cv=True, iter_num=1, n_trees=200, cv=None, n_fold=5, 
             metrics = ('accuracy', 'f1_weighted', 'precision_weighted', 'recall_weighted'), **kwargs):
    "Fitting RF models and rturning the models and their cross-validation scores."
    results = {}

    if regres:
        fitted_model = skensemble.RandomForestRegressor(n_estimators=n_trees)
    else:
        fitted_model = skensemble.RandomForestClassifier(n_estimators=n_trees)
    
    fitted_model = fitted_model.fit(df, y)
    results['model'] = fitted_model

    # Setting up variables for imp_feat storing
    imp_feat = np.zeros((iter_num * n_fold, len(df.columns)))
    f = 0

    if not return_cv:
        return(fitted_model)
    if cv is None:
        cv = sklearn.model_selection.StratifiedKFold(n_fold, shuffle=True)

    if type(metrics) is str:
        stores_res = {metrics:[]}
    else:
        store_res = {m:[] for m in metrics}

    for _ in range(iter_num):
        if regres:
            rf = skensemble.RandomForestRegressor(n_estimators=n_trees)
        else:
            rf = skensemble.RandomForestClassifier(n_estimators=n_trees)
        
        cv_res = sklearn.model_selection.cross_validate(rf, df, y, cv=cv, scoring=metrics, **kwargs)
        
        if type(metrics) is str:
            print(stores_res)
            store_res[metrics].extend(cv_res['test_'+metrics])
        else:
            for i in metrics:
                store_res[i].extend(cv_res['test_'+i])

        for train_index, test_index in cv.split(df, y):
            # Random Forest setup and fit
            if regres:
                rf = skensemble.RandomForestRegressor(n_estimators=n_trees)
            else:
                rf = skensemble.RandomForestClassifier(n_estimators=n_trees)
            X_train, X_test = df.iloc[train_index, :], df.iloc[test_index, :]
            y_train, y_test = [y[i] for i in train_index], [y[i] for i in test_index]
            rf.fit(X_train, y_train)

            # Compute important features
            imp_feat[f, :] = rf.feature_importances_ # Importance of each feature
            f = f + 1

    # Collect and order all important features values from each Random Forest
    imp_feat_sum = imp_feat.sum(axis=0) / (iter_num * n_fold)
    results['imp_feat'] = sorted(enumerate(imp_feat_sum), key=lambda x: x[1], reverse=True)

    results.update(store_res)
    return results#{'model': fitted_model, 'cv_scores': scores}

In [ ]:
# Choose a number for the seed for consistent results
np.random.seed()

n_trees=200 # Number of trees in the model

RF_results = metsta.RF_model(treated_data, target, regression, # Data, labels and if it's a regression or classification
                return_cv=True, iter_num=5, # If you want cross validation results and number of iterations for it
                n_trees=n_trees, # Number of trees in the model
                cv=None, n_fold=3, # Choose a method of cross-validation (None is stratified cv) and the number of folds
    
        # For Classification Problems
         metrics = ('accuracy', 'f1_weighted', 'precision_weighted', 'recall_weighted')) # Choose the performance metrics

        # For Regression problems
        #metrics = ('neg_mean_squared_error',), n_jobs=-1)

Performance analysis

In [ ]:
rf_results_summary = pd.DataFrame(columns=['Value', 'Standard Deviation'])
for k,v in RF_results.items():
    if k != 'model' and k != 'imp_feat':
        rf_results_summary.loc[k] = np.mean(v), np.std(v)

print(rf_results_summary)

**Important Feature analysis**

See the most important features for class discrimination (sorted by importance).

In [ ]:
imp_feats_rf = meta_data.copy()
imp_feats_rf.insert(0,'Bucket label', imp_feats_rf.index)
imp_feats_rf.insert(1,'Gini Importance', '')
for n in range(len(RF_results['imp_feat'])):
    imp_feats_rf['Gini Importance'].iloc[RF_results['imp_feat'][n][0]] = RF_results['imp_feat'][n][1]
imp_feats_rf = imp_feats_rf.sort_values(by='Gini Importance', ascending=False)
imp_feats_rf.index = range(1, len(imp_feats_rf)+1)

In [ ]:
imp_feats_rf.head(20) # Select number of features to see

In [ ]:
# Saving Important feature dataset in an excel
SAVE_IMP_FEAT = False

# Saving the most important features by their fraction 'frac_feat_impor'.
# If None, saving the most important features based on a threshold 'VIP_Score_threshold'.
# If also None, save the full dataset of all features
frac_feat_impor = 0.02 # Fraction of features to save, If None the variable in the next line is used.
score_threshold = None # Only used if variable above is None, threshold of score to consider a feature important.

if SAVE_IMP_FEAT:
    if frac_feat_impor:
        max_idx = int(frac_feat_impor*len(imp_feats_rf))
        filt_imp_feats_rf = imp_feats_rf.iloc[:max_idx]
        filt_imp_feats_rf.to_excel(f'RF_ImpFeat_{frac_feat_impor*100}%.xlsx')
    elif score_threshold:
        filt_imp_feats_rf = imp_feats_rf[imp_feats_rf['Gini Importance'] > score_threshold]
        filt_imp_feats_rf.to_excel(f'RF_ImpFeat_GiniImpgreater{score_threshold}.xlsx')
    else:
        imp_feats_rf.to_excel(f'RF_FeatByImportance.xlsx')

### RF Permutation Test

This is a test to observe if the model performance is significant, that is, if it is better than a random model. If it is, then the remaining results from the important features give meaningful information, if not, then you cannot use the important features results since they essentially mean nothing.

The permutation test will permutate the class labels of your samples, that is, all classes will be randomized while maintaining the same number of samples per class and classes. Then, for each permutation it will see the model performance. 

The default metric for model performance is `accuracy`. If you have an imbalanced model, accuracy is not a good metric, so you should change to another such as `f1_weighted`.

**Note: Permutation tests take a while to do, thus the default is False in the begginning so you can make a first analysis on your dataset. If you then want to use the results of a supervised model, run a permutation test to check if your model is significant.**

p-value calculation: (1 + nº of times permutated model has better performance than non-permutated model)/nº of permutations.

In [ ]:
GENERATE = False # True if you want to do, False if not
if GENERATE:
    # Set a random seed for reproducibility of cross validation
    np.random.seed()
    # (Random seed of labels permutations is in the random permutator)

    perm_results_RF = metsta.permutation_RF(
        treated_data, target, regression,  # data, labels and if it's a regression
        iter_num=500, # Nº of permutations to do in your test - around 500 should be enough
        n_trees=200, # Number of trees in the model
        cv=None, n_fold=3, # Choose a method of cross-validation (None is stratified cv) and the number of folds
        random_state=None, # Random seed given to make the permutations rng class labels
        metric=('accuracy')) # Choose a metric to use to evaluate if the model is significant

In [ ]:
if GENERATE:
    with plt.style.context('seaborn-whitegrid'):
        fig, ax = plt.subplots(1,1, figsize=(6,6))

        n_labels = len(treated_data.index)
        tab20bcols = sns.color_palette('tab20b', 20)
        perm_results = perm_results_RF
        
        # Histogram with performance of permutated values
        hist_res = ax.hist(np.array(perm_results[1]), n_labels, range=(0, 1.00001), label='RF Permutations',
                     edgecolor='black', color=tab20bcols[1], alpha = 1)
        
        # Plot the non-permutated model performance
        ylim = [0, hist_res[0].max()*1.2]
        ax.plot(2 * [perm_results[0]], ylim, '-', linewidth=3, color='darkred', #alpha = 0.5,
                     label='p-value %.5f)' % perm_results[2], solid_capstyle='round')
        ax.tick_params(labelsize=13)
        ax.set_xlabel('CV Model Performance', fontsize=14)
        ax.set_ylabel('Nº of occurrences', fontsize=14)
        if perm_results[0] >= 0.5:
            ax.text(perm_results[0]-0.45, hist_res[0].max()*1.1, 'p-value = %.3f' % perm_results[2], fontsize = 15)
        else:
            ax.text(perm_results[0]+0.05, hist_res[0].max()*1.1, 'p-value = %.3f' % perm_results[2], fontsize = 15)
        ax.set_title('Random Forest Permutation Test', size = 15)
        #ax.grid()
        ax.set_axisbelow(True)

        #fig.savefig('Name_RF_PermutationTest.jpg', dpi=400) # Save the Figure

### ROC curves (Receiver Operating Characteristic)

This basically gives you an area under curve that the closer it is to 1, the better our model. We also iterate this n_iter times so we have a softer curve and to give as a better indication of the actual area under curve (AUC). This plots the true positive rate against the false positive rate.

**Only possible for when your datasets have 2 classes. Choose the class which is considered the 'positive' class.**

Credit to initial function to the BinSim paper.

If you do not have 2 classes, skip ahead this section.

In [ ]:
GENERATE = True
if GENERATE:
    if regression:
        print('You are working on a regression problem. Thus, ROC curves are not made.')
    else:
        if len(pd.unique(target)) == 2:
            # Set a random seed for reproducibility
            np.random.seed()
            
            # Set up positive label
            pos_label = pd.unique(target)[0]

            resROC_RF = metsta.RF_ROC_cv(treated_data, target, regres=regression, # Data, target and if it's a regression
                                        pos_label=pos_label, # Positive label
                                        n_trees=200, # Number of trees of RF
                                        n_iter=15, # Number of iterations to repeat 
                                        cv=None, n_fold=3) # Method of CV (None is stratified cv) and the number of folds
        else:
            print('Your target has more than 2 classes. Thus, ROC curves are not made.')

In [ ]:
if GENERATE:
    if len(pd.unique(target)) == 2:
        # Plot the ROC curves 
        with sns.axes_style("whitegrid"):
            with sns.plotting_context("notebook", font_scale=1.2):
                f, ax = plt.subplots(1, 1, figsize=(5,5), constrained_layout=True)
                res = resROC_RF
                mean_fpr = res['average fpr']
                mean_tpr = res['average tpr']
                mean_auc = res['mean AUC']
                mean_fpr = [0,] + list(mean_fpr)
                mean_tpr = [0,] + list(mean_tpr)
                ax.plot(mean_fpr, mean_tpr,
                       label=f'AUC = {mean_auc:.3f}',
                       lw=2, alpha=0.8)
                ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='lightgrey', alpha=.8)
                ax.legend()
                ax.set_xlim(None, 1)
                ax.set_ylim(0, None)
                ax.set_title('Random Forest ROC Curve', fontsize=15)

                #f.savefig('Name_RF_ROCcurve.jpg', dpi=400) # Save the figure

## Step 5.2: PLS-DA (Partial Least Squares - Discriminant Analysis) <a class="anchor" id="step-5_2"></a>

**Back to [Table of Contents](#toc)**

First, an optimization of the number of components of PLS-DA and a **set of functions for PLS-DA - `optim_PLSDA_n_components` for example - to see in metanalysis_standard.**

The VIPs scores are calculated using the function `_calculate_vips` in multianalysis.py that comes from the link https://www.researchgate.net/post/How-can-I-compute-Variable-Importance-in-Projection-VIP-in-Partial-Least-Squares-PLS as provided by Keiron Teilo O'Shea in that link.

**Note: `max_comp` (maximum number of components) cannot be higher than the number of samples that will train a model minus 1. For example, if you have 15 samples and a 3-fold cross-validation each fold will have 5 samples. A training set will be comprised of two of those folds thus it will have 10 samples, thus `max_comp` (and `n_comp` later on) cannot be higher than 9. Another example if you have 22 samples and 5 folds, the folds will have 4/4/4/5/5 samples each. A training set will have four of these folds and the minimum sum of them is 4+4+4+5-1=16, thus max_comp cannot be higher than 16.**

In [ ]:
%%capture --no-stdout
# above is to supress PLS warnings

# Set the random seed
np.random.seed()

max_comp = 9 # Max. number of components to search (the higher the more time it takes)

# Store Results
PLS_optim = metsta.optim_PLSDA_n_components(treated_data, target, regression, # Data, target and if it's a regression
                                    encode2as1vector=True,
                                    max_comp=max_comp, # Max. number of components to search
                                    kf=None, n_fold=3, # Cross validation to use (none is stratified CV) and nº of folds
                                    scale=False) # Set scale to True only if you did not do scaling in pre-treatments

In the figure below, $R^{2}$ and $Q^{2}$ are shown. You want to choose the number of components **where $Q^{2}$ specifically** stops increasing, so, in this case, 4 components will be chosen. 

- $Q^{2}$ - PLS score by its mean squared error based on the test samples, thus it is ideal to test if the model will overfit. This will increase until a certain number of components that should be chosen. Then it usually stabilizes but from a certain point it might start to decrease which would mean the model is overfitting. For example, in this case, we choose 4 components based on this score, but you could choose 5 or 6 and it would not affect the model a lot.
- $R^{2}$ - PLS score by its mean squared error based on the training samples used to make the model (it will be higher than $Q^{2}$ but it should not be used to choose the number of components. This metric always increases with the more components used which means it will overfit the model eventually.

In [ ]:
scores_cols = sns.color_palette('tab10', 10) # Set the colors for the lines
with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1.2):
        f, ax = plt.subplots(1, 1, figsize=(5,5), constrained_layout=True) # Set the figure size
        c = 0
        for i, values in PLS_optim.items():
            if i =='CVscores':
                name = 'Q$^2$'
            else:
                name = 'R$^2$'
            
            ax.plot(range(1, len(values) + 1), values, label=name, color = scores_cols[c])
            c = c+1
        
        ax.set(xlabel='Number of Components', # Set the label for the x axis
                ylabel='PLS Score') # Set the label for the Y axis
        ax.legend(loc='lower right', fontsize=15) # Set the legend
        ax.set_ylim([0, 1.02]) # Set limits for y axis
        ax.set_xticks(range(0, len(values), 2)) # Set ticks that appear in the bottom of x axis
        plt.show()

### PLS-DA model fitting

**See details of `PLSDA_model_cv` function (model fitting AND evaluation) in metanalysis_standard.py as adapted from the one in the BinSim paper.**

The VIPs scores are calculated using the function `_calculate_vips` in multianalysis.py that comes from the link https://www.researchgate.net/post/How-can-I-compute-Variable-Importance-in-Projection-VIP-in-Partial-Least-Squares-PLS as provided by Keiron Teilo O'Shea in that link.

The function `_generate_y_PLSDA` is also present in multianalysis.py.

In [ ]:
%%capture --no-stdout
# above is to supress PLS warnings

n_comp = 9 # Number of components of PLS-DA model - very important

PLSDA_results = metsta.PLSDA_model_CV(treated_data, target, regression, # Data, target and if it's a regression
                       n_comp=n_comp, # Number of components of PLS-DA model - very important
                       kf = None, n_fold=3, # Cross validation to use (none is stratified CV) and nº of folds
                       iter_num=10, # Number of iterations of cross-validation to do
                       encode2as1vector=True,
                       scale=False, # Set scale to True only if you did not do scaling in pre-treatments
                       feat_type='VIP') # Feature Importance Metric to use, default is VIP scores (see function for others)

**Performance analysis**

In [ ]:
pls_results_summary = pd.DataFrame(columns=['Value', 'Standard Deviation'])
for k,v in PLSDA_results.items():
    if k != 'Q2' and k != 'imp_feat':
        pls_results_summary.loc[k] = np.mean(v), np.std(v)

print(pls_results_summary)

**Important Feature analysis**

See the most important features for class discrimination.

In [ ]:
imp_feats_plsda = meta_data.copy()
imp_feats_plsda.insert(0,'Bucket label', imp_feats_plsda.index)
imp_feats_plsda.insert(1,'VIP Score', '')
for n in range(len(PLSDA_results['imp_feat'])):
    imp_feats_plsda['VIP Score'][PLSDA_results['imp_feat'][n][0]] = PLSDA_results['imp_feat'][n][1]
imp_feats_plsda = imp_feats_plsda.sort_values(by='VIP Score', ascending=False)
imp_feats_plsda.index = range(1, len(imp_feats_plsda)+1)

In [ ]:
imp_feats_plsda.head(20) # Select number of features to see

In [ ]:
# Saving Important feature dataset in an excel
SAVE_IMP_FEAT = False

# Saving the most important features by their fraction 'frac_feat_impor'.
# If None, saving the most important features based on a threshold 'VIP_Score_threshold'.
# If also None, save the full dataset of all features
frac_feat_impor = 0.02 # Fraction of features to save, If None the variable in the next line is used.
VIP_Score_threshold = 1 # Only used if variable above is None, threshold of score to consider a feature important.

if SAVE_IMP_FEAT:
    if frac_feat_impor:
        max_idx = int(frac_feat_impor*len(imp_feats_plsda))
        filt_imp_feats_plsda = imp_feats_plsda.iloc[:max_idx]
        filt_imp_feats_plsda.to_excel(f'PLSDA_ImpFeat_{frac_feat_impor*100}%.xlsx')
    elif VIP_Score_threshold:
        filt_imp_feats_plsda = imp_feats_plsda[imp_feats_plsda['VIP Score'] > VIP_Score_threshold]
        filt_imp_feats_plsda.to_excel(f'PLSDA_ImpFeat_VIPgreater{VIP_Score_threshold}.xlsx')
    else:
        imp_feats_plsda.to_excel(f'PLSDA_FeatByImportance.xlsx')

### Sample Projection on the two most important Components/Latent Variables of PLS models 

**To do** See if it's worth doing this in a regression

In [ ]:
if not regression:
    n_components = 4 # Nº of componentes

    model, scores = fit_PLSDA_model(treated_data, target,
                                    n_comp=n_components, scale=False, # Only true if scaling was not done earlier
                                    encode2as1vector=True,
                                    lv_prefix='LV ', label_name='Label')

    lcolors = label_colours

    with sns.axes_style("whitegrid"):
        with sns.plotting_context("notebook", font_scale=1.2):
            fig, ax = plt.subplots(1,1, figsize=(6,6)) # Set up fig size
            plot_PCA(scores, lcolors, title="PLS Projection", ax=ax,
                    components=(1,2)) # Select components to see
            plt.title('PLS Projection', fontsize=20) # Title
            plt.legend(loc='upper right', ncol=1, fontsize=15)  # Legend           
            plt.tight_layout()
            plt.show()
            
            #fig.savefig('Name_PLSplot.jpg', dpi=400) # Save the figure

### PLS-DA Permutation Test

This is a test to observe if the model performance is significant, that is, if it is better than a random model. If it is, then the remaining results from the important features give meaningful information, if not, then you cannot use the important features results since they essentially mean nothing.

The permutation test will permutate the class labels of your samples, that is, all classes will be randomized while maintaining the same number of samples per class and classes. Then, for each permutation it will see the model performance. 

The default metric for model performance is `accuracy`. If you have an imbalanced model, accuracy is not a good metric, so you should change to another such as `f1_weighted`. Metric can only be: `accuracy`, `f1_weighted`, `recall_weighted` or `precision_weighted`.

**Note: Permutation tests take a while to do, thus the default is False in the begginning so you can make a first analysis on your dataset. If you then want to use the results of a supervised model, run a permutation test to check if your model is significant.**

p-value calculation: (1 + nº of times permutated model has better performance than non-permutated model)/nº of permutations.

In [ ]:
GENERATE = True # True if you want to do, False if not
if GENERATE:
    # Set a random seed for reproducibility of cross validation
    np.random.seed()
    # (Random seed of labels permutations is in the random state in the function below)

    perm_results_PLSDA = metsta.permutation_PLSDA(
        treated_data, target,  # data and labels
        n_comp=4, # Number of components
        iter_num=500, # Nº of permutations to do in your test - around 500 should be enough
        cv=None, n_fold=3, # Choose a method of cross-validation (None is stratified cv) and the number of folds
        random_state=None, # Random seed given to make the permutations rng class labels
        encode2as1vector=True, scale=False, # Set scale to True only if you did not do scaling in pre-treatments
        metric='accuracy') # Choose a metric to use to evaluate if the model is significant

In [ ]:
if GENERATE:
    with plt.style.context('seaborn-v0_8-whitegrid'):
        fig, ax = plt.subplots(1,1, figsize=(6,6))

        n_labels = len(treated_data.index)
        tab20bcols = sns.color_palette('tab20b', 20)
        perm_results = perm_results_PLSDA
        
        # Histogram with performance of permutated values
        hist_res = ax.hist(np.array(perm_results[1]), n_labels, range=(0, 1.00001), label='PLS-DA Permutations',
                     edgecolor='black', color=tab20bcols[1], alpha = 1)
        
        # Plot the non-permutated model performance
        ylim = [0, hist_res[0].max()*1.2]
        ax.plot(2 * [perm_results[0]], ylim, '-', linewidth=3, color='darkred', #alpha = 0.5,
                     label='p-value %.5f)' % perm_results[2], solid_capstyle='round')
        ax.tick_params(labelsize=13)
        ax.set_xlabel('CV Model Performance', fontsize=14)
        ax.set_ylabel('Nº of occurrences', fontsize=14)
        if perm_results[0] >= 0.5:
            ax.text(perm_results[0]-0.45, hist_res[0].max()*1.1, 'p-value = %.3f' % perm_results[2], fontsize = 15)
        else:
            ax.text(perm_results[0]+0.05, hist_res[0].max()*1.1, 'p-value = %.3f' % perm_results[2], fontsize = 15)
        ax.set_title('PLS-DA Permutation Test', size = 15)
        #ax.grid()
        ax.set_axisbelow(True)

        #fig.savefig('Name_PLSDA_PermutationTest.jpg', dpi=400) # Save the Figure

### ROC curves (Receiver Operating Characteristic)

This basically gives you an area under curve that the closer it is to 1, the better our model. We also iterate this n_iter times so we have a softer curve and to give as a better indication of the actual area under curve (AUC). This plots the true positive rate against the false positive rate.

**Only possible for when your datasets have 2 classes. Choose the class which is considered the 'positive' class.**

Credit to initial function to the BinSim paper.

If you do not have 2 classes, skip ahead this section.

In [ ]:
GENERATE = True
if GENERATE:
    if regression:
        print('You are working on a regression problem. Thus, ROC curves are not made.')
    else:
        if len(pd.unique(target)) == 2:
            # Set a random seed for reproducibility
            np.random.seed()
            
            # Set up positive label
            pos_label = pd.unique(target)[0]

            resROC_PLSDA = metsta.PLSDA_ROC_cv(treated_data, target, # Data and target
                                pos_label=pos_label, # Positive label
                                n_comp=4, # Number of components
                                scale=False, # Set scale to True only if you did not do scaling in pre-treatments
                                n_iter=15, # Number of iterations to repeat 
                                cv=None, n_fold=3) # method of cross-validation (None is stratified cv) and the number of folds
        else:
            print('Your target has more than 2 classes. Thus, ROC curves are not made.')

In [ ]:
# Plot the ROC curves 
if GENERATE:
    if len(pd.unique(target)) == 2:
        with sns.axes_style("whitegrid"):
            with sns.plotting_context("notebook", font_scale=1.2):
                f, ax = plt.subplots(1, 1, figsize=(5,5), constrained_layout=True)
                res = resROC_PLSDA
                mean_fpr = res['average fpr']
                mean_tpr = res['average tpr']
                mean_auc = res['mean AUC']
                mean_fpr = [0,] + list(mean_fpr)
                mean_tpr = [0,] + list(mean_tpr)
                ax.plot(mean_fpr, mean_tpr,
                       label=f'AUC = {mean_auc:.3f}',
                       lw=2, alpha=0.8)
                ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='lightgrey', alpha=.8)
                ax.legend()
                ax.set_xlim(None, 1)
                ax.set_ylim(0, None)
                ax.set(xlabel='False positive rate', ylabel='True positive rate')
                ax.set_title('PLS-DA ROC Curve', fontsize=15)

                #f.savefig('Name_PLSDA_ROCcurve.jpg', dpi=400) # Save the figure

## Step 5.3: XGBoost (eXtreme Gradient Boosting) <a class="anchor" id="step-5_3"></a>

**Back to [Table of Contents](#toc)**

Thhis block of code automatically selects an XGBoost objective function for your specific use case. If you want to use a different function, you may select it here, or in the 'objective' input to the functions.

Some reading on objective functions:
- https://xgboost.readthedocs.io/en/stable/parameter.html (Ctrl-F objective)
- https://machinelearningmastery.com/xgboost-loss-functions/

In [ ]:
xgb_analysis = False

In [ ]:
if regression:
    objective = "reg:squarederror"
else:
    if len(pd.unique(target)) == 2:
        print('Warning: XGBoost is currently unreliable for binary classification tasks. If you still want to use it delete xgb_analysis = False')
        objective = "binary:logistic"
        xgb_analysis = False
    else:
        objective = "multi:softprob"

We first start with a brief optimization of the parameters for XGBoost training. Default is to focus only on the number of estimators (trees) and their maximum depth. However, there are other parameters that can be tweaked, simply by adding new terms to the xgb_optim_params dictionary. Please be aware that each new parameter will explonentially increase the running time of the function, and that for regression problem even just a single-parameter tuning can take very long.

To 'fix' an hyperparater that you do not want to tune at a non-default value, simply add it to the XGB_optim function as **kwargs

Resources on XGBoost Hyperparameters and their tuning:
- https://xgboost.readthedocs.io/en/stable/parameter.html
- https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning
- https://freedium.cfd/https://towardsdatascience.com/xgboost-fine-tune-and-optimize-your-model-23d996fab663

In [ ]:
if xgb_analysis:
    # Select a random seed (number between the ()) if you don't want the results to change every time you run the code
    np.random.seed()

    xgb_max_n_estimators = 300

    xgb_optim_params = {'n_estimators': range(10,xgb_max_n_estimators+1,5)} 

    #xgb_optim_params = {'min_child_weight': numeric_range(0,1,0.1), 'subsample': numeric_range(0,1,0.1), 'gamma': numeric_range(0,1,0.1), 
    #                    'max_depth': range(0,10,1)}

    #XGB_Optim = metsta.optimise_xgb_parameters(treated_data, target, xgb_optim_params, regression, objective)
    XGB_Optim = metsta.optimise_xgb_parameters(treated_data, target, xgb_optim_params, regression, objective, n_estimators=200)

In [ ]:
if xgb_analysis:
    param_to_plot = 'n_estimators'

    # Plotting the results and adjusting parameters of the plot
    with sns.axes_style("whitegrid"):
        with sns.plotting_context("notebook", font_scale=1.2):
            f, ax = plt.subplots(1, 1, figsize=(6,6), constrained_layout=True) # Set Figure Size

            c_map = sns.color_palette('tab10', 10)

            ax.plot(XGB_Optim.cv_results_['param_n_estimators'], [s*100 for s in XGB_Optim.cv_results_['mean_test_score']])
            ax.set_ylabel('XGBoost CV Mean Accuracy (%)', fontsize=15) # Set the y_label and size
            ax.set_xlabel(param_to_plot, fontsize=15)
            ax.set_title('XGBoost', fontsize=18) # Set the title and size
            ax.set_ylim([30,101]) # Set the limits on the y axis

            #f.suptitle('Optimization of the number of trees')
            ax.legend(fontsize=15) # Set the legend and size
            plt.show()

### Fitting the XGBoost model

You may add more parameters to the function as **kwargs

https://xgboost.readthedocs.io/en/stable/parameter.html

In [ ]:
if xgb_analysis:

    n_estimators = 200

    XGB_results = metsta.XGB_model(treated_data, target, # Data and labels
                    regres=regression, obj=objective, # Regression or classification, and objective function
                    return_cv=True, iter_num=5, # If you want cross validation results and number of iterations for it
                    n_estimators=n_estimators, # Number of trees in the model
                    cv=None, n_fold=3, # Choose a method of cross-validation (None is stratified cv) and the number of folds
                    #metrics = ('neg_mean_squared_error', 'r2'), subsample=0.7)
                    metrics = ('accuracy', 'f1_weighted', 'precision_weighted', 'recall_weighted')) #, gamma=0, min_child_weight=0.9, subsample=0.4), # Choose the performance metrics

In [ ]:
if xgb_analysis:

    results_summary = pd.DataFrame(columns=['Value', 'Standard Deviation'])
    for k,v in XGB_results.items():
        if k != 'model' and k != 'imp_feat':
            results_summary.loc[k] = np.mean(v), np.std(v)

    print(results_summary)

In [ ]:
if xgb_analysis:
    imp_feats_xgb = meta_data.copy()
    imp_feats_xgb.insert(0,'Bucket label', imp_feats_xgb.index)
    imp_feats_xgb.insert(1,'Feature Importance', '')
    for n in range(len(XGB_results['imp_feat'])):
        imp_feats_xgb['Feature Importance'].iloc[XGB_results['imp_feat'][n][0]] = XGB_results['imp_feat'][n][1]
    imp_feats_xgb = imp_feats_xgb.sort_values(by='Feature Importance', ascending=False)
    imp_feats_xgb.index = range(1, len(imp_feats_xgb)+1)
else:
    imp_feats_xgb = 'XGBoost analysis was not performed'
imp_feats_xgb

### XGBoost Permutation tests

In [ ]:
GENERATE=False
if GENERATE:
    # Set a random seed for reproducibility of cross validation
    np.random.seed()
    # (Random seed of labels permutations is in the random permutator)

    perm_results_XGB = metsta.permutation_XGB(
        treated_data, target,  # data and labels
        regres=regression, obj=objective, # regression vs classification and objective function 
        iter_num=100, # Nº of permutations to do in your test - around 500 should be enough
        n_estimators=200, # Number of trees in the model
        cv=None, n_fold=3, # Choose a method of cross-validation (None is stratified cv) and the number of folds
        random_state=None, # Random seed given to make the permutations rng class labels
        metric=('accuracy')) # Choose a metric to use to evaluate if the model is significant

In [ ]:
if GENERATE:
    with plt.style.context('seaborn-v0_8-whitegrid'):
        fig, ax = plt.subplots(1,1, figsize=(6,6))

        n_labels = len(treated_data.index)
        tab20bcols = sns.color_palette('tab20b', 20)
        perm_results = perm_results_XGB
        
        # Histogram with performance of permutated values
        hist_res = ax.hist(np.array(perm_results[1]), n_labels, range=(0, 1.00001), label='XGBoost Permutations',
                     edgecolor='black', color=tab20bcols[1], alpha = 1)
        
        # Plot the non-permutated model performance
        ylim = [0, hist_res[0].max()*1.2]
        ax.plot(2 * [perm_results[0]], ylim, '-', linewidth=3, color='darkred', #alpha = 0.5,
                     label='p-value %.5f)' % perm_results[2], solid_capstyle='round')
        ax.tick_params(labelsize=13)
        ax.set_xlabel('CV Model Performance', fontsize=14)
        ax.set_ylabel('Nº of occurrences', fontsize=14)
        if perm_results[0] >= 0.5:
            ax.text(perm_results[0]-0.45, hist_res[0].max()*1.1, 'p-value = %.3f' % perm_results[2], fontsize = 15)
        else:
            ax.text(perm_results[0]+0.05, hist_res[0].max()*1.1, 'p-value = %.3f' % perm_results[2], fontsize = 15)
        ax.set_title('XGBoost Permutation Test', size = 15)
        #ax.grid()
        ax.set_axisbelow(True)

        #fig.savefig('Name_XGB_PermutationTest.jpg', dpi=400) # Save the Figure

### ROC Curves (Reveicer Operating Characteristics)

In [ ]:
GENERATE = False
if GENERATE:
    if regression:
        print('You are working on a regression problem. Thus, ROC curves are not made.')
    else:
        if len(pd.unique(target)) == 2:
            # Set a random seed for reproducibility
            np.random.seed()
            
            # Set up positive label
            pos_label = pd.unique(target)[0]

            resROC_RF = metsta.XGB_ROC_cv(treated_data, target, # Data and target
                                        pos_label=pos_label, obj=objective, # Positive label and objective
                                        n_estimators=200, # Number of trees of RF
                                        n_iter=15, # Number of iterations to repeat 
                                        cv=None, n_fold=3) # Method of CV (None is stratified cv) and the number of folds
        else:
            print('Your target has more than 2 classes. Thus, ROC curves are not made.')

# Step 6: Univariate Analysis and Fold-Change Analysis <a class="anchor" id="step-6"></a>

**Back to [Table of Contents](#toc)**

In this section, both Univariate Analysis and Fold-Change analysis are performed outputting a DataFrame ordered by lowest _p_-value to highest and with the columns of Fold change and logarithmic Fold Change.

The Fold change is calculated in a dataset with missing values imputed and normalized after. **This means that with our very high number of missing values in FT-ICR-MS data, it affects the calculation of the fold change a lot. Thus, take this fold changes values with a grain (or multiple grains that are actually more like rocks than grains) of salt.**

Choose between the parametric **t-test** and non-parametric **Mann-Whitney test** for 2-class univariate analysis, and between **ANOVA** and **Kruskal-Wallis test** for multiclass analysis.

The functions for 2-class analysis all come from the multianalysis.py file (from the BinSim paper) but most of them were slightly altered and put in the metanalysis_standard.py file to be able to specify a control and test class.

## Step 6.1: 2-class Univariate Statistical Analysis <a class="anchor" id="step-6_1"></a>

**Back to [Table of Contents](#toc)**

Choose **control** class

In [ ]:
control_class = classes[0]
control_class

Choose **_p_-value significancy** threshold. Usual values are: 0.05, 0.01, 0.001. If you **do not want** to filter the data based on a _p_-value threshold, make alpha=**None**.

In [ ]:
alpha = None
alpha = 0.05 # If you use Mann-Whitney with low number of samples you might have to change this alpha
# It can be the correct appraoch with a lot of missing values though
# With Mann-Whitney test, there will be a set of discrete p-values as well

Choose **Fold change** threshold value.

If you want to only select from the significant features, those that have a fold change greater than X, change **abs_log2FC_threshold**. This value is in **log 2 of the absolute fold change**. For example, if you want to only consider features that have a fold change greater than 2-fold (2 or 0.5), then the abs_log2FC_threshold should be 1. If you want it to be greater than 3-fold, the threshold should be np.log2(3).

In [ ]:
abs_log2FC_threshold = 1 # As a default, I will choose not to perform this step
# Example for 2-fold threshold: abs_log2FC_threshold = 1 or np.log2(2)
# Example for 3-fold threshold: abs_log2FC_threshold = np.log2(3)

**Perform Univariate Analysis**

If you have more than 2 classes, the pre-treatment must be equal to the pre-treatment made in step 2, thus we use the same variables as before (hence the importance of choosing the pre-treatment by the variables and not in the function itself in step 2.

If the filtering chosen is `total_samples` and the filt_kw is greater than 1, that is, a minimum number of samples to appear in the dataset, that number is transformed to represent the same percentage of samples in the smaller subsection of data of only the control and test classes. E.g. if the minimum nº of samples a feature must appear in a 15-sample dataset is 4, then by performing univariate analysis between 2 classes on a 6-sample subset, the minimum nº of samples allowed is (4/15) * 6 = 1.6 rounded up, that is, it has to appear in at least 2 of that 6 sample-subset (and 4 samples of the original 15).

In [ ]:
useMW = False # Consider variance between groups as equal
equal_var = True # Use Mann-Whitney Test or standard T-test

if len(classes) == 2:
    test_class = [cl for cl in classes if cl != control_class][0]
    # Perform Univariate Analysis
    univariate_results = metsta.compute_FC_pvalues_2groups(normalized=univariate_data, # Used for Fold-Change Computation
                                  processed=treated_data, # Used for p-value computation
                                  labels=target, # Labels of the samples
                                  control_class=control_class, # Control class
                                  test_class=test_class, # Non-control class
                                  equal_var=useMW, # Consider variance between groups as equal
                                  useMW=useMW) # Use Mann-Whitney Test or standard T-test
    
    # Select only Features considered significative
    if alpha:
        filt_uni_results = univariate_results[univariate_results['FDR adjusted p-value'] < alpha].copy()
    else:
        filt_uni_results = univariate_results.copy()
    
    # Select features that have an absolute fold change (in log2) greater than abs_log2FC_threshold
    if abs_log2FC_threshold:
        # Calculate absolute Log2 Fold-Change
        filt_uni_results['abs_log2FC'] = abs(filt_uni_results['log2FC'])
        # Select
        filt_uni_results = filt_uni_results[filt_uni_results['abs_log2FC'] > abs_log2FC_threshold]
        filt_uni_results = filt_uni_results.drop(columns='abs_log2FC')
        
    print('Univariate Analysis Done.')

# More than 2 classes
else:
    test_classes = [cl for cl in classes if cl != control_class]
    univariate_results = {}
    filt_uni_results = {}
    univariate_df = {}

    for test_class in test_classes: # For each non-control class
        # Select only the samples of the control and current test class
        selection = [i in [control_class, test_class] for i in target]
        target_temp = list(np.array(target)[selection])
        
        file_temp = annotated_data[sample_cols].copy()
        file_temp = file_temp.loc[:, selection]

        # Perform the same filtering and pre-treatments steps but using only the control and test class samples

        # Transform the filt_kw if needed to match the filtering done above (as explained in the markdown cell above)
        if filt_method == 'total_samples':
            # Adapting the filt_kw to a smaller subset of samples
            # Use % of the original filtering used to calculate the equivalent number of samples in subset and round UP
            # Possible Issue - since we already used the filtered dataset (because it has annotations and de-duplications),
            # the data filtering with 'total_samples' is not perfect - a feature must pass this data filtering but also
            # the original data filtering made
            if filt_kw > 1:
                f_kw = math.ceil(filt_kw/len(sample_cols)*sum(selection))
            else:
                f_kw = filt_kw
        else:
            f_kw = filt_kw

        t_data,_,filt_data,_,_ = metsta.filtering_pretreatment(
                          file_temp, list(np.array(target)[selection]), file_temp.columns,
           #### Everything here must be the same as in step 2 except the f_kw as explained above
                          filt_method, f_kw, extra_filt, mvi, mvi_kw, norm, norm_kw, tf, tf_kw, scaling, scaling_kw)

        univariate_df[test_class] = [t_data, target_temp]
        
        # Perform Univariate Analysis on this newly acquired data
        univariate_results[test_class] = metsta.compute_FC_pvalues_2groups(
                                  normalized=filt_data, # Used for Fold-Change Computation
                                  processed=t_data, # Used for p-value computation
                                  labels=target_temp, # Labels of the samples
                                  control_class=control_class, # Control class
                                  test_class=test_class, # Non-control class
                                  equal_var=equal_var, # Consider variance between groups as equal
                                  useMW=useMW) # Use Mann-Whitney Test if True or standard T-test if False
        
        # Select only Features considered significative
        if alpha:
            filt_uni_results[test_class] = univariate_results[test_class][univariate_results[test_class][
                'FDR adjusted p-value'] < alpha].copy()
        else:
            filt_uni_results[test_class] = univariate_results[test_class].copy()
        
        # Select features that have an absolute fold change (in log2) greater than abs_log2FC_threshold
        if abs_log2FC_threshold:
            # Calculate absolute Log2 Fold-Change
            filt_uni_results[test_class]['abs_log2FC'] = abs(filt_uni_results[test_class]['log2FC'])
            # Select
            filt_uni_results[test_class] = filt_uni_results[test_class][
                filt_uni_results[test_class]['abs_log2FC'] > abs_log2FC_threshold]
            filt_uni_results[test_class] = filt_uni_results[test_class].drop(columns='abs_log2FC')

        print(f'Univariate Analysis {test_class} vs. {control_class} Done.')

In [ ]:
# Add a column with the meta data of each feature
temp_meta_data = meta_data[[i for i in meta_data.columns if i not in ['Neutral Mass', 'Probable m/z', 'm/z']]]

# If you have 2 classes
if len(classes) == 2:
    filt_uni_results = pd.concat((filt_uni_results, temp_meta_data.loc[filt_uni_results.index]), axis=1)

# More than 2 classes
else:
    for test_class in filt_uni_results.keys():
        filt_uni_results[test_class] = pd.concat((filt_uni_results[test_class], temp_meta_data.loc[
            filt_uni_results[test_class].index]), axis=1)

Get the results in an Excel if you want to

In [ ]:
Univariate_Excel = False # Change to True to have the Excels
if Univariate_Excel:
    if len(classes) == 2:
        filt_uni_results.to_excel(f'UniAnalysis_pvalue{alpha}_log2FC{abs_log2FC_threshold}.xlsx')
    else:
        for i in filt_uni_results:
            filt_uni_results[i].to_excel(f'UniAnalysis_class{i}_pvalue{alpha}_log2FC{abs_log2FC_threshold}.xlsx')

### Example Results (for more than 2 classes, a random class was chosen)

**This is usually just useful for 2 classes, we will leave here the visualizations for more than 2 classes as an example.** Using more than 2 classes in analysis would require the use of all possibile combinations instead of just looking at the 2 of them. For example, seeing the features which are significant in at least one of the cases.

The first four columns have the results of the univariate analysis.

In [ ]:
if len(classes) == 2:
    example_results = filt_uni_results
    example_heatmap = treated_data
    example_target = target
    example_nonfilt_res = univariate_results
else:
    example_results = filt_uni_results[test_classes[0]]
    example_heatmap = univariate_df[test_classes[0]][0]
    example_target = univariate_df[test_classes[0]][1]
    example_nonfilt_res = univariate_results[test_classes[0]]

See results ordered by **_p_-value**

In [ ]:
example_results.head(20)

See biggest fold changes from the test class in relation to the control class (1st cell) and vice-versa (2nd cell)

In [ ]:
example_results.sort_values(by='log2FC', ascending=False).head(20)

In [ ]:
example_results.sort_values(by='log2FC', ascending=False).tail(20)

In [ ]:
example_nonfilt_res

**Heatmap** of the example shown considering only the significant features from the univariate analysis (can be better)

In [ ]:
# Simple heatmap comparing intensities in treated_data
f, ax = plt.subplots(1,1, figsize=(5, 7), constrained_layout=True) # Set figure size
sns.heatmap(example_heatmap.T.loc[example_results.index],
            cmap='RdBu_r', # Select colormap to use
            vmin=-3, vmax=3) # Adjust minimum and maximum values in the Heatmap colorbar
ax.tick_params(left=False)
ax.set_yticklabels('')
ax.set_ylabel('')
plt.show()

**Volcano Plot** of the example shown considering only the significant features from the univariate analysis

**Much more useful when there are only 2 classes.**

In [ ]:
f, ax = plt.subplots(1,1, figsize=(7, 6), constrained_layout=True) # Set figure size

alpha # Threshold used for p-values
abs_log2FC_threshold # Threshold used for fold changes

non_sig_feat_color = 'silver'
reduced_sig_color = 'deepskyblue'
increased_sig_color = 'lightcoral'

non_sig_feats = []
increased_sig_feats = []
reduced_sig_feats = []
for i in example_nonfilt_res.index:
    if i not in example_results.index:
        non_sig_feats.append(i)
    elif example_nonfilt_res.loc[i, 'log2FC'] < 0:
        reduced_sig_feats.append(i)
    else:
        increased_sig_feats.append(i)

ax.scatter(example_nonfilt_res.loc[non_sig_feats, 'log2FC'], 
           -np.log10(example_nonfilt_res.loc[non_sig_feats, 'FDR adjusted p-value']),
           c=non_sig_feat_color, alpha=0.7, label='Non-Significant')
ax.scatter(example_nonfilt_res.loc[reduced_sig_feats, 'log2FC'], 
            -np.log10(example_nonfilt_res.loc[reduced_sig_feats, 'FDR adjusted p-value']),
            c=reduced_sig_color, alpha=0.7, label='Downregulated')
ax.scatter(example_nonfilt_res.loc[increased_sig_feats, 'log2FC'], 
            -np.log10(example_nonfilt_res.loc[increased_sig_feats, 'FDR adjusted p-value']),
            c=increased_sig_color, alpha=0.7, label='Upregulated')

ax.axhline(-np.log10(alpha), color='black', linestyle='--')
if abs_log2FC_threshold != None:
    ax.axvline(abs_log2FC_threshold, color='black', linestyle='--')
    ax.axvline(-abs_log2FC_threshold, color='black', linestyle='--')

ax.set_ylabel('- log$_1$$_0$(Adjusted (Benjamini-Hochberg) p-value)', fontsize=14)
t_class = pd.unique(example_target)[pd.unique(example_target) != control_class][0]
ax.set_xlabel(f'log$_2$(Fold Change))', fontsize=14)
ax.set_title(f'Volcano Plot - {t_class}/{control_class}', fontsize=18)
ax.legend(fontsize=12, bbox_to_anchor=(1,1))
#f.savefig('Name_UniAnalysis_VolcanoPlot.jpg', dpi=400)
plt.show()

## Step 6.2: Multi-class Univariate Statistical Analysis <a class="anchor" id="step-6_2"></a>

**Back to [Table of Contents](#toc)**

In [ ]:
if len(classes) == 2:
    multiclass_univariate_results = 'Your data has only two classes'
else:
    multiclass_univariate_results = metsta.compute_pvalues_multiple_groups(treated_data, groups, 
                                            useKW=False) # Choose if you want to use Kruskal-Wallis test (non-parametric)

In [ ]:
multiclass_univariate_results

# Step 7: Make Van Krevelen Diagrams, Kendrick Mass Defect Plots and Chemical Composition series for your samples <a class="anchor" id="step-7"></a>

**Back to [Table of Contents](#toc)**

In [ ]:
SmartFormula = True # Is there a SmartFormula Annotation? Change to False if not
#  If you have Smart Fromula, do you also want to use formulas from the annotations performed here?
include_other_formula_cols = False

if SmartFormula:
    formula_subset = ["Formula",]
    if include_other_formula_cols:
        formula_subset.extend(meta_cols_formulas)
else:
    formula_subset = meta_cols_formulas
print(formula_subset)

**Van Krevelen Plots**

See the options for **color_dots_by** in the beginning of the next cell.

This section plots a Van Krevelen Plot for each of the classes under analysis. This is made by only considering metabolites
(features) that appear at least in one sample of said class. Only metabolites with assigned formulas are considered and the columns with formulas considered were chosen in the previous cell (**Note: you HAVE to select at least one annotation**). **If multiple formulas can be assigned to a metabolite whether within the same database annotation or different, they are both considered and plotted in the Van Krevelen.**

In [ ]:
# Van Krevelen Diagrams
color_dots_by = 'Rank' # Other option 'logInt'
# Van Krevelen points (peaks) are coloured based on their average intensity
# Rank - colored by the rank of their average intensity compared to others
# logInt - colored by the logarithm of their averaged intensity compared to others

midpoint = 0.75 # Marks the point where the color passes from the low intensity color to the high intensity
# 0.75 means 75% of points will be coloured with the low intensity color (stronger the lesser the intensity)

for g in group_dfs:
    # Calculating H/C and O/C ratios
    forms = group_dfs[g].dropna(subset=formula_subset, how='all')
    elems = create_element_counts(forms, formula_subset=formula_subset)

    f, ax = plt.subplots(1,1, figsize=(6,6)) # Setting axes for the figs
    
    # Make the vector that will govern the colour of the plots
    if color_dots_by == 'Rank':
        c = univariate_data[elems.index].loc[np.array(target) == g].mean().rank(ascending=False)
        slope_midpoint = midpoint*len(univariate_data[elems.index].columns)
    elif color_dots_by == 'logInt':
        c = np.log(univariate_data[elems.index].loc[np.array(target) == g].mean())
        slope_midpoint = c.sort_values().iloc[int(midpoint*len(elems.index))]
    
    if formula_subset == 'Formula':
        y = elems['H/C'].loc[c.index]
        x = elems['O/C'].loc[c.index]
    else:
        y = elems['H/C']
        x = elems['O/C']
    
    plt.scatter(x, y, s=3, c=c, cmap='bwr', norm=TwoSlopeNorm(slope_midpoint))
    plt.xlabel('O/C', fontsize=20)
    plt.ylabel('H/C', fontsize=20)
    ax.margins(y=.1)
    ax.set_ylim([0.15,2.01])
    ax.set_xlim([-0.1,2.01])
    plt.colorbar()
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_title(g, fontsize=20)
    
    
    #f, ax = plt.subplots(1,1, figsize=(8,8)) # Setting axes for the figs
    #plt.title(g, fontsize=20)
    
    #f.savefig('Name_VKplot_' + g + '.jpg', dpi=400) # Save the figure

**Kendrick Mass Defect Plots**

See the options for **rounding** in the beginning of the next cell.

On cases where you have **multiple formulas belonging to different chemical composition series assigned to the same _m/z_ peak**, points are marked as Ambiguous (Amb.) which has the same colour as 'No Formula Assigned' (No Form.).

In [ ]:
rounding = 'up' # 'up' or 'nearest' - how Kendrick Nominal Mass is obtained by rounding up or to the nearest integer

# Put these two lines instead of the current if you want to try to put them all in the same fig, you have to adjust things
# for that to take into especially how many panels you need (right now it is (2,3) that is 2*3=6) and the figsize.
#f, axs = plt.subplots(2,3, figsize=(16,12))
#for g, ax in zip(group_dfs, axs.ravel()):
for g in group_dfs:
    f, ax = plt.subplots(1,1, figsize=(7,6), constrained_layout=True) # Setting axes for the figs
    
    # Getting the classes each formula has
    forms = group_dfs[g].dropna(subset=formula_subset, how='all')
    elems = create_element_counts(forms, formula_subset=formula_subset)
    
    l = []
    n_form_per_peak = pd.Series(elems.index).value_counts()

    for i in group_dfs[g].index: # For each peak to consider
        # If it has formula assigned by the formula columns selected
        if i in elems.index:
            # See if it has more than one formula
            if n_form_per_peak.loc[i] > 1:
                # If it has, see if they belong to the same class series or not
                cl = set(elems.loc[i, 'Series'])
                if len(cl) == 1: # If yes assign it
                    l.append(elems.loc[i, 'Series'].iloc[0])
                else: # If not, say it is Ambiguous
                    l.append('No Form. / Amb.')

            # In case it only has one formula
            else:
                l.append(elems['Series'].loc[i])

        # If it has not formula assigned by the formula columns selected
        else:
            l.append('No Form. / Amb.')
    classes_series = ['CHO', 'CHOS', 'CHON', 'CHNS', 'CHONS', 'CHOP', 'CHONP','CHONSP', 'other', 'No Form. / Amb.']
    dict_col = {lbl: c for lbl, c in zip(classes_series, sns.color_palette('tab10', len(classes_series)))}
    list_col = np.array([dict_col[i] for i in l])
    
    # Calculate points for the scatter plot, choose if rounding shouls happen to the nearest        
    nominal, fraction = metsta.calc_kmd(group_dfs[g], rounding=rounding)

    # Scatter plot
    for cl in classes_series:
        n = np.array(nominal)[np.array(l) == cl]
        f = np.array(fraction)[np.array(l) == cl]
        lc = list_col[np.array(l) == cl]
        scatter = ax.scatter(n,f, s=6, c=lc, label=cl)

    ax.set_xlabel('Kendrick Nominal Mass', fontsize=20) # Set X axis label
    ax.set_ylabel('Kendrick Mass Defect', fontsize=20) # Set Y axis label
    ax.set_xlim([0,1250])
    ax.legend(fontsize=12, loc='upper left', bbox_to_anchor=(1,1)) # Set legend
    ax.set_title(g, fontsize= 20) # Set title
    
    #f.savefig('Name_KMDplot_' + g + '.jpg', dpi=400) # Save the figure

plt.show()

**Chemical Composition Series**

**If multiple formulas can be assigned to the same _m/z_ peak** whether within the same formula annotation made or between different annotations, **each one will be counted in this plot**. That is, if a peak in a class has 3 possible candidate formulas, 2 belonging to the 'CHO' series and another to the 'CHOP' series; then 2 formulas will be added to the 'CHO' series and 1 to the 'CHOP' series. Thus, we are considering that the 3 elementary formulas are represented by that peak (probably an overestimation). In all cases, it is rare to find multiple candidate formulas for the same m/z peak, especially with extreme-resolution data.

In [ ]:
def plot_composition_series(group_dfs,
                            series_order=('CHO', 'CHOS', 'CHON', 'CHNS', 'CHONS', 'CHOP', 'CHONP','CHONSP', 'other'),
                            formula_col=formula_subset,
                            label_colours=label_colours,
                            ax=None):
    # Chemical compostition series
    bar_number = len(group_dfs)
    curr_num = - len(group_dfs)/2 + 0.5
    width = 0.8/bar_number - 0.05
    series = pd.DataFrame()
    
    for g in group_dfs:
        forms = group_dfs[g].dropna(subset=formula_subset, how='all')
        elems = create_element_counts(forms, formula_subset=formula_subset)
        #print('This is the chemical composition Series for', g, ':\n')
    
        counts = elems['Series'].value_counts().reindex(series_order)
        series[g] = counts # Store the counts
        x = np.arange(len(counts))  # the label locations
        ax.barh(x+(width+0.05)*curr_num, counts, width, label=g, color=label_colours[g])
        curr_num = curr_num + 1
    ax.set_yticks(x)
    ax.set_yticklabels(counts.index)
    ax.set_xlabel('Nº of Formulas', fontsize = 15)
    ax.set_title('Chemical Composition Series', fontsize=15)
    plt.grid(zorder=0)
    plt.legend()
    
    return series

In [ ]:
f, ax = plt.subplots(1,1, figsize=(9,6)) # Setting axes for the figs
chem_comp_series = plot_composition_series(group_dfs, ax=ax,label_colours=label_colours,)
#f.savefig('Name_CCSeries.jpg', dpi=400) # Save the figure
plt.show()

In [ ]:
chem_comp_series.T

#### PCA of chemical composition series per sample with Loading arrows

In [ ]:
# Calculating the chemical composition series for each samples
series = pd.DataFrame()
series_order=('CHO', 'CHOS', 'CHON', 'CHNS', 'CHONS', 'CHOP', 'CHONP','CHONSP', 'other')
for sample in sample_cols:
    forms = processed_data.dropna(subset=sample).dropna(subset=formula_subset, how='all')
    elems = create_element_counts(forms, formula_subset=formula_subset)

    counts = elems['Series'].value_counts().reindex(series_order)
    series[sample] = counts # Store the counts
series = series.replace({np.nan:0})
series

In [ ]:
def plot_PCA_loading_arrows(series, principaldf, loadings, var_exp,
                            n_components, top_features=None, method='top_variance', ax=None):
    """Draw Loading arrows of top n features.
     
     Based on idea and taken from St. Ovf. thread (Qiyun Zhu answer).
     https://stackoverflow.com/questions/39216897/plot-pca-loadings-and-loading-in-biplot-in-sklearn-like-rs-autoplot."""
    
    if ax is None:
        ax = plt.gca()
    
    # Taken from Stack Overflow
    if top_features:
        # Method 1: Find top arrows that appear the longest (i.e., furthest from the origin) in the visible plot
        if method == 'top_longest':
            tops = (loadings ** 2).sum(axis=1).argsort()[-top_features:]
            loadings_to_plot = loadings[tops]
            idxs = np.array(series.index)[tops]

        # Method 2: Find top features that drive most variance in the visible PCs:
        elif method == 'top_variance':
            tops = (np.abs(loadings) * var_exp).sum(axis=1).argsort()[-top_features:]
            loadings_to_plot = loadings[tops]
            idxs = np.array(series.index)[tops]
        
        else:
            raise ValueError('Method is not accepted. It should be one of "top_longest" or "top_variance".')
            
        # Scale the loadings so they are easier to interpret sicne their absolute values have no grand meaning
        loadings_to_plot /= np.sqrt((loadings_to_plot ** 2).sum(axis=0))
        loadings_to_plot = loadings_to_plot * np.array(np.abs(principaldf.iloc[:,:n_components]).max(axis=0))

        # Empirical formula to determine arrow width (according to St. Ovf. Answer)
        width = -0.005 * np.min([np.subtract(*plt.xlim()), np.subtract(*plt.ylim())])
            
        # Draw arrows
        for arrow in range(top_features):
            ax.arrow(0,0, loadings_to_plot[arrow,0], loadings_to_plot[arrow,1], alpha=0.7, ec='none', width=width,
                     color='grey', length_includes_head=True)
            ax.text(loadings_to_plot[arrow,0]*1.04, loadings_to_plot[arrow,1]*1.04, idxs[arrow],
                     ha='center', va='center')
        return
    
    # Scale the loadings so they are easier to interpret sicne their absolute values have no grand meaning
    loadings_to_plot = loadings * np.array(np.abs(principaldf.iloc[:,:n_components]).max(axis=0))
    
    # Empirical formula to determine arrow width (according to St. Ovf. Answer) but thinner
    width = -0.005 * np.min([np.subtract(*plt.xlim()), np.subtract(*plt.ylim())])
    
    # Draw arrows
    for arrow in range(len(series)):
        ax.arrow(0,0, loadings_to_plot[arrow,0], loadings_to_plot[arrow,1], alpha=0.7, ec='none', width=width,
                 color='grey', length_includes_head=True)
        ax.text(loadings_to_plot[arrow,0]*1.04, loadings_to_plot[arrow,1]*1.04, series.index[arrow],
                 ha='center', va='center')

In [ ]:
# Calculating PCA
n_components = 2 # Select number of components to calculate
principaldf, var, loadings = metsta.compute_df_with_PCs_VE_loadings(transf.pareto_scale(series.T), 
                                       n_components=n_components, # Select number of components to calculate
                                       whiten=True, labels=target, return_var_ratios_and_loadings=True)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(6,6)) # Change the size of the figure
# Plot PCA
ax.axis('equal')
lcolors = label_colours

plot_PCA(principaldf, lcolors, 
         components=(1,2), # Select components to see
         title='', # Select title of plot
         ax=ax)

# Remove ellipses by putting a # before the next line
plot_ellipses_PCA(principaldf, 
                  lcolors, 
                  components=(1,2), # Select components to see
                  ax=ax, 
                  q=0.95) # Confidence ellipse with 95% (q) confidence

# Putting loading arrows
plot_PCA_loading_arrows(series, principaldf, loadings, var, n_components, top_features=None, ax=ax)

ax.set_xlabel(f'PC 1 ({var[0] * 100:.1f} %)', size=15) # Set the size of labels
ax.set_ylabel(f'PC 2 ({var[1] * 100:.1f} %)', size=15) # Set the size of labels

plt.legend(fontsize=15, loc='upper left', bbox_to_anchor=(1,1)) # Set the size of labels
#plt.grid() # If you want a grid or not
plt.show()
#f.savefig('Name_CCS_PCAplot.png', dpi=400) # Save the figure

# Step 8: Pathways Assignment of HMDB Annotated Metabolites <a class="anchor" id="step-8"></a>

**Back to [Table of Contents](#toc)**

This section reads a pathways database build based on Ramp (https://academic.oup.com/bioinformatics/article/39/1/btac726/6827287) to use as a base to assign the pathways each compound belongs to.

**It only works with HMDB identifiers and HMDB annotated compounds.**

The result is a DataFrame where the index is each HMDB identifier in the `Matched_HMDB_IDs` columns of your data and the two columns represent the assigned pathways and corresponding IDs from the different databases RampID uses plus and extra column with the corresponding HMDB metabolite name.

In [ ]:
# Load pathway database into a DataFrame
with open('RAMP_ID_pathways_improved.pickle', 'rb') as handle:
    pathways_db = pickle.load(handle)
pathways_db = pathways_db[['HMDB Name', 'Pathway Name', 'Pathway ID']]
pathways_db

In [ ]:
# Make pathways assignment
pathways_assignment_idx = []
pathways_assignment_name = []
if 'Matched HMDB IDs' in processed_data.columns:
    for i in processed_data['Matched HMDB IDs'].dropna().index:
        for hmdb_id in range(len(processed_data.loc[i, 'Matched HMDB IDs'])):
            if processed_data.loc[i, 'Matched HMDB IDs'][hmdb_id] in pathways_assignment_idx:
                continue
            else:
                pathways_assignment_idx.append(processed_data.loc[i, 'Matched HMDB IDs'][hmdb_id])
                pathways_assignment_name.append(processed_data.loc[i, 'Matched HMDB names'][hmdb_id])

In [ ]:
# Build the pathways assignment dataframe
pathways_assignment = pd.DataFrame(index=pathways_assignment_idx, columns=['Pathway Name', 'Pathway ID'])
if 'Matched HMDB IDs' in processed_data.columns:
    for i in pathways_assignment.index:
        if i in pathways_db.index:
            pathways_assignment.loc[i] = pathways_db.loc[i]
    pathways_assignment['Matched HMDB names'] = pathways_assignment_name

In [ ]:
# See full pathways assignment dataframe
pathways_assignment

In [ ]:
# See only the HMDB that led to pathway assignments
pathways_assignment.dropna()

Select chosen list of HMDB you want to search

In [ ]:
chosen_hmdbs = ['HMDB0000045', 'HMDB0009947', 'HMDB0000001']

filtered_chosen_hmdbs = []
for i in chosen_hmdbs:
    if i not in pathways_assignment.index:
        print(f'{i} was not annotated for the currently analysed data and is thus not in the pathway assignment DataFrame.')
    else:
        filtered_chosen_hmdbs.append(i)

pathways_assignment.loc[filtered_chosen_hmdbs]

# Step 9: KEGG Colour Mapping <a class="anchor" id="step-9"></a>

**Back to [Table of Contents](#toc)**

In [ ]:
class_1_colour = 'red'
class_2_colour = 'blue'
both_colour = 'purple'

print('Present only in',classes[0]+':',class_1_colour)
print('Present only in',classes[1]+':',class_2_colour)
print('Present in both', class_1_colour, 'and', class_1_colour+':', both_colour)

In [ ]:
if len(pd.unique(target)) == 2:

    kegg_data = annotated_data.copy()
    kegg_data = kegg_data.dropna(subset='Matched HMDB IDs')
    kegg_data = kegg_data[sample_cols+['Matched HMDB IDs']]
    kegg_data = kegg_data.explode('Matched HMDB IDs', ignore_index=True)
    kegg_data['Matched KEGG'] = np.nan
    for a in tqdm(kegg_data.index):
        kegg_hmdb_id = kegg_data['Matched HMDB IDs'][a]
        kegg_data['Matched KEGG'][a] = dbs['HMDB']['DB'].loc[kegg_hmdb_id]['kegg']
    kegg_data = kegg_data.dropna(subset='Matched KEGG')
    kegg_data['Colour'] = np.nan
    for k in kegg_data.index:
        k_df = kegg_data.loc[[k]]
        if k_df[groups[classes[0]]].isnull().values.all():
            kegg_data['Colour'][k] = class_2_colour
        elif k_df[groups[classes[1]]].isnull().values.all():
            kegg_data['Colour'][k] = class_1_colour
        else:
            kegg_data['Colour'][k] = both_colour
    kegg_data
else:
    print('Your data has more than two classes. Thus, KEGG colour mapping is not performed')

In [ ]:
SAVE_KEGG_COLOURS = True
if len(pd.unique(target)) == 2:
    kegg_colours = kegg_data[['Matched KEGG', 'Colour']]
    kegg_colours = kegg_colours.set_index('Matched KEGG')
    if SAVE_KEGG_COLOURS:
        kegg_colours_filename = filename.split('.')[0]+'_kegg_colours.csv'
        kegg_colours.to_csv(kegg_colours_filename, sep="\t")

# Step 10: BinSim Specific Analysis <a class="anchor" id="step-10"></a>

**Back to [Table of Contents](#toc)**

All analysis done very quickly using BinSim
- PCA
- HCA
- Random Forest
- PLS-DA
- XGBoost

BinSim was performed as described in the BinSim paper as mentioned earlier.

PCA

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(6,6)) # Change the size of the figure

principaldf_BinSim, var_BinSim, loadings_BinSim = metsta.compute_df_with_PCs_VE_loadings(bin_data, 
                                       n_components=2, # Select number of components to calculate
                                       whiten=True, labels=target, return_var_ratios_and_loadings=True)
# Plot PCA
ax.axis('equal')
lcolors = label_colours
plot_PCA(principaldf_BinSim, lcolors, 
         components=(1,2), # Select components to see
         title='', # Select title of plot
         ax=ax)

# Remove ellipses by putting a # before the next line
plot_ellipses_PCA(principaldf_BinSim, lcolors, 
                  components=(1,2), # Select components to see
                  ax=ax, 
                  q=0.95) # Confidence ellipse with 95% (q) confidence

ax.set_xlabel(f'PC 1 ({var_BinSim[0] * 100:.1f} %)', size=15) # Set the size of labels
ax.set_ylabel(f'PC 2 ({var_BinSim[1] * 100:.1f} %)', size=15) # Set the size of labels

plt.legend(fontsize=15) # Set the size of labels
plt.grid() # If you want a grid or not
plt.show()
#f.savefig('Name_BinSim_PCAplot.png', dpi=400) # Save the figure

HCA

In [ ]:
# Performing HCA 
metric = 'euclidean' # Select distance metric
method = 'ward' # Select linkage method

distances_BinSim = dist.pdist(bin_data, metric=metric)
Z_BinSim = hier.linkage(distances_BinSim, method=method)

hca_res_BinSim = {'Z': Z_BinSim, 'distances': distances_BinSim}

# Plot HCA
with sns.axes_style("white"):
    f, ax = plt.subplots(1, 1, figsize=(4, 4), constrained_layout=True) # Set Figure Size
    plot_dendogram(hca_res_BinSim['Z'], 
                   target, ax=ax,
                   label_colors=label_colours,
                   title='', # Select title
                   color_threshold=0) # Select a distance threshold from where different sets of lines are coloured

    plt.show()
    #f.savefig('Name_BinSim_HCAplot.png', dpi=400) # Save the figure

Random Forest

In [ ]:
# Choose a number for the seed for consistent results
np.random.seed()

n_trees=200 # Number of trees in the model

RF_results_BinSim = metsta.RF_model(bin_data, target, regres=regression, # Data and labels 
                      return_cv=True, iter_num=5, # If you want cross calidation results and number of iterations for it
                      n_trees=n_trees, # Number of trees in the model
                      cv=None, n_fold=3, # Choose a method of cross-validation (None is stratified cv) and the nº of folds
        metrics = ('accuracy', 'f1_weighted', 'precision_weighted', 'recall_weighted')) # Choose the performance metrics

rf_results_summary_BinSim = pd.DataFrame(columns=['Value', 'Standard Deviation'])
for k,v in RF_results_BinSim.items():
    if k != 'model' and k != 'imp_feat':
        rf_results_summary_BinSim.loc[k] = np.mean(v), np.std(v)

print(rf_results_summary_BinSim)

imp_feats_rf_BinSim = meta_data.copy()
imp_feats_rf_BinSim.insert(0,'Bucket label', imp_feats_rf_BinSim.index)
imp_feats_rf_BinSim.insert(1,'Gini Importance', '')
imp_feats_rf_BinSim = imp_feats_rf_BinSim.copy()
for n in range(len(RF_results_BinSim['imp_feat'])):
    imp_feats_rf_BinSim['Gini Importance'].iloc[RF_results_BinSim['imp_feat'][n][0]] = RF_results_BinSim['imp_feat'][n][1]
imp_feats_rf_BinSim = imp_feats_rf_BinSim.sort_values(by='Gini Importance', ascending=False)
imp_feats_rf_BinSim.index = range(1, len(imp_feats_rf_BinSim)+1)

In [ ]:
imp_feats_rf_BinSim.head(20)

In [ ]:
# Saving Important feature dataset in an excel
SAVE_IMP_FEAT = False

# Saving the most important features by their fraction 'frac_feat_impor'.
# If None, saving the most important features based on a threshold 'VIP_Score_threshold'.
# If also None, save the full dataset of all features
frac_feat_impor = 0.02 # Fraction of features to save, If None the variable in the next line is used.
score_threshold = None # Only used if variable above is None, threshold of score to consider a feature important.

if SAVE_IMP_FEAT:
    if frac_feat_impor:
        max_idx = int(frac_feat_impor*len(imp_feats_rf_BinSim))
        filt_imp_feats_rf_BinSim = imp_feats_rf_BinSim.iloc[:max_idx]
        filt_imp_feats_rf_BinSim.to_excel(f'RF_BinSim_ImpFeat_{frac_feat_impor*100}%.xlsx')
    elif score_threshold:
        filt_imp_feats_rf_BinSim = imp_feats_rf_BinSim[imp_feats_rf_BinSim['Gini Importance'] > score_threshold]
        filt_imp_feats_rf_BinSim.to_excel(f'RF_BinSim_ImpFeat_GiniImpgreater{score_threshold}.xlsx')
    else:
        imp_feats_rf_BinSim.to_excel(f'RF_BinSim_FeatByImportance.xlsx')

In [ ]:
GENERATE = False # True if you want to do, False if not
if GENERATE:
    # Set a random seed for reproducibility of cross validation
    np.random.seed()
    # (Random seed of labels permutations is in the random permutator)

    perm_results_BinSim_RF = metsta.permutation_RF(
        bin_data, target,  # Data and labels 
        iter_num=500, # Nº of permutations to do in your test - around 500 should be enough
        n_trees=200, # Number of trees in the model
        cv=None, n_fold=3, # Choose a method of cross-validation (None is stratified cv) and the number of folds
        random_state=None, # Random seed given to make the permutations rng class labels
        metric=('accuracy')) # Choose a metric to use to evaluate if the model is significant
    
    with plt.style.context('seaborn-whitegrid'):
        fig, ax = plt.subplots(1,1, figsize=(6,6))

        n_labels = len(bin_data.index)
        tab20bcols = sns.color_palette('tab20b', 20)
        perm_results = perm_results_BinSim_RF
        
        # Histogram with performance of permutated values
        hist_res = ax.hist(np.array(perm_results[1]), n_labels, range=(0, 1.00001), label='RF Permutations',
                     edgecolor='black', color=tab20bcols[1], alpha = 1)
        
        # Plot the non-permutated model performance
        ylim = [0, hist_res[0].max()*1.2]
        ax.plot(2 * [perm_results[0]], ylim, '-', linewidth=3, color='darkred', #alpha = 0.5,
                     label='p-value %.5f)' % perm_results[2], solid_capstyle='round')
        ax.tick_params(labelsize=13)
        ax.set_xlabel('CV Model Performance', fontsize=14)
        ax.set_ylabel('Nº of occurrences', fontsize=14)
        if perm_results[0] >= 0.5:
            ax.text(perm_results[0]-0.45, hist_res[0].max()*1.1, 'p-value = %.3f' % perm_results[2], fontsize = 15)
        else:
            ax.text(perm_results[0]+0.05, hist_res[0].max()*1.1, 'p-value = %.3f' % perm_results[2], fontsize = 15)
        ax.set_title('Random Forest Permutation Test (BinSim)', size = 15)
        #ax.grid()
        ax.set_axisbelow(True)

        #fig.savefig('Name_BinSim_RF_PermutationTest.jpg', dpi=400) # Save the Figure

In [ ]:
GENERATE = True
if GENERATE:
    if len(pd.unique(target)) == 2:
        # Set a random seed for reproducibility
        np.random.seed()
        
        # Set up positive label
        pos_label = pd.unique(target)[0]

        resROC_BinSim_RF = metsta.RF_ROC_cv(bin_data, target, regression, # Data and target
                                    pos_label=pos_label, # Positive label
                                    n_trees=200, # Number of trees of RF
                                    n_iter=15, # Number of iterations to repeat 
                                    cv=None, n_fold=3) # method of CV (None is stratified cv) and the number of folds
    
        with sns.axes_style("whitegrid"):
            with sns.plotting_context("notebook", font_scale=1.2):
                f, ax = plt.subplots(1, 1, figsize=(5,5), constrained_layout=True)
                res = resROC_BinSim_RF
                mean_fpr = res['average fpr']
                mean_tpr = res['average tpr']
                mean_auc = res['mean AUC']
                mean_fpr = [0,] + list(mean_fpr)
                mean_tpr = [0,] + list(mean_tpr)
                ax.plot(mean_fpr, mean_tpr,
                       label=f'AUC = {mean_auc:.3f}',
                       lw=2, alpha=0.8)
                ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='lightgrey', alpha=.8)
                ax.legend()
                ax.set_xlim(None, 1)
                ax.set_ylim(0, None)
                ax.set_title('Random Forest ROC Curve (BinSim)', fontsize=15)

                #f.savefig('Name_BinSim_RF_ROCcurve.jpg', dpi=400) # Save the figure
    else:
        print('Your target has more than 2 classes. Thus, ROC curves are not made.')

PLS-DA

In [ ]:
# Store Results
PLS_optim_BinSim = metsta.optim_PLSDA_n_components(bin_data, target, regression, # Data and target
                                    encode2as1vector=True,
                                    max_comp=8, # Max. number of components to search (the higher the more time it takes)
                                    kf=None, n_fold=3, # Cross validation to use (none is stratified CV) and nº of folds
                                    scale=False) # Set scale to True only if you did not do scaling in pre-treatments

scores_cols = sns.color_palette('tab10', 10) # Set the colors for the lines
with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1.2):
        f, ax = plt.subplots(1, 1, figsize=(4,4), constrained_layout=True) # Set the figure size
        c = 0
        for i, values in PLS_optim.items():
            if i =='CVscores':
                name = 'Q$^2$'
            else:
                name = 'R$^2$'
            
            ax.plot(range(1, len(values) + 1), values, label=name, color = scores_cols[c])
            c = c+1
        
        ax.set(xlabel='Number of Components', # Set the label for the x axis
                ylabel='PLS Score') # Set the label for the Y axis
        ax.legend(loc='lower right', fontsize=15) # Set the legend
        ax.set_ylim([0, 1.02]) # Set limits for y axis
        ax.set_xticks(range(0, len(values), 2)) # Set ticks that appear in the bottom of x axis
        plt.show()

In [ ]:
#%%capture --no-stdout
# above is to supress PLS warnings

n_comp = 4 # Number of components of PLS-DA model - very important

PLSDA_results_BinSim = metsta.PLSDA_model_CV(bin_data, target, regression, # Data and target
                       n_comp=n_comp, # Number of components of PLS-DA model - very important
                       kf=None, n_fold=3, # Cross validation to use (none is stratified CV) and nº of folds
                       iter_num=10, # Number of iterations of cross-validation to do
                       encode2as1vector=True,
                       scale=False, # Set scale to True only if you did not do scaling in pre-treatments
                       feat_type='VIP') # Feature Importance Metric to use, default is VIP scores (see function for others)

pls_results_summary_BinSim = pd.DataFrame(columns=['Value', 'Standard Deviation'])
for k,v in PLSDA_results_BinSim.items():
    if k != 'Q2' and k != 'imp_feat':
        pls_results_summary_BinSim.loc[k] = np.mean(v), np.std(v)

print(pls_results_summary_BinSim)

imp_feats_plsda_BinSim = meta_data.copy()
imp_feats_plsda_BinSim.insert(0,'Bucket label', imp_feats_plsda_BinSim.index)
imp_feats_plsda_BinSim.insert(1,'VIP Score', '')
for n in range(len(PLSDA_results_BinSim['imp_feat'])):
    imp_feats_plsda_BinSim['VIP Score'][PLSDA_results_BinSim['imp_feat'][n][0]] = PLSDA_results_BinSim['imp_feat'][n][1]
imp_feats_plsda_BinSim = imp_feats_plsda_BinSim.sort_values(by='VIP Score', ascending=False)
imp_feats_plsda_BinSim.index = range(1, len(imp_feats_plsda_BinSim)+1)

imp_feats_plsda_BinSim.head(20) # Select number of features to see

In [ ]:
# Saving Important feature dataset in an excel
SAVE_IMP_FEAT = False

# Saving the most important features by their fraction 'frac_feat_impor'.
# If None, saving the most important features based on a threshold 'VIP_Score_threshold'.
# If also None, save the full dataset of all features
frac_feat_impor = 0.02 # Fraction of features to save, If None the variable in the next line is used.
VIP_Score_threshold = 1 # Only used if variable above is None, threshold of score to consider a feature important.

if SAVE_IMP_FEAT:
    if frac_feat_impor:
        max_idx = int(frac_feat_impor*len(imp_feats_plsda_BinSim))
        filt_imp_feats_plsda_BinSim = imp_feats_plsda_BinSim.iloc[:max_idx]
        filt_imp_feats_plsda_BinSim.to_excel(f'PLSDA_BinSim_ImpFeat_{frac_feat_impor*100}%.xlsx')
    elif VIP_Score_threshold:
        filt_imp_feats_plsda_BinSim = imp_feats_plsda_BinSim[imp_feats_plsda_BinSim['VIP Score'] > VIP_Score_threshold]
        filt_imp_feats_plsda_BinSim.to_excel(f'PLSDA_BinSim_ImpFeat_VIPgreater{VIP_Score_threshold}.xlsx')
    else:
        imp_feats_plsda_BinSim.to_excel(f'PLSDA_BinSim_FeatByImportance.xlsx')

In [ ]:
GENERATE = False # True if you want to do, False if not
if GENERATE:
    # Set a random seed for reproducibility of cross validation
    np.random.seed()
    # (Random seed of labels permutations is in the random state in the function below)

    perm_results_BinSim_PLSDA = metsta.permutation_PLSDA(
        bin_data, target,  # data and labels
        n_comp=4, # Number of components
        iter_num=500, # Nº of permutations to do in your test - around 500 should be enough
        cv=None, n_fold=3, # Choose a method of cross-validation (None is stratified cv) and the number of folds
        random_state=None, # Random seed given to make the permutations rng class labels
        encode2as1vector=True, scale=False, # Set scale to True only if you did not do scaling in pre-treatments
        metric='accuracy') # Choose a metric to use to evaluate if the model is significant
    
    with plt.style.context('seaborn-whitegrid'):
        fig, ax = plt.subplots(1,1, figsize=(6,6))

        n_labels = len(treated_data.index)
        tab20bcols = sns.color_palette('tab20b', 20)
        perm_results = perm_results_BinSim_PLSDA
        
        # Histogram with performance of permutated values
        hist_res = ax.hist(np.array(perm_results[1]), n_labels, range=(0, 1.00001), label='PLS-DA Permutations',
                     edgecolor='black', color=tab20bcols[1], alpha = 1)
        
        # Plot the non-permutated model performance
        ylim = [0, hist_res[0].max()*1.2]
        ax.plot(2 * [perm_results[0]], ylim, '-', linewidth=3, color='darkred', #alpha = 0.5,
                     label='p-value %.5f)' % perm_results[2], solid_capstyle='round')
        ax.tick_params(labelsize=13)
        ax.set_xlabel('CV Model Performance', fontsize=14)
        ax.set_ylabel('Nº of occurrences', fontsize=14)
        if perm_results[0] >= 0.5:
            ax.text(perm_results[0]-0.45, hist_res[0].max()*1.1, 'p-value = %.3f' % perm_results[2], fontsize = 15)
        else:
            ax.text(perm_results[0]+0.05, hist_res[0].max()*1.1, 'p-value = %.3f' % perm_results[2], fontsize = 15)
        ax.set_title('PLS-DA Permutation Test (BinSim)', size = 15)
        #ax.grid()
        ax.set_axisbelow(True)

        #fig.savefig('Name_PLSDA_PermutationTest.jpg', dpi=400) # Save the Figure

In [ ]:
GENERATE = True
if GENERATE:
    if len(pd.unique(target)) == 2:
        # Set a random seed for reproducibility
        np.random.seed()
        
        # Set up positive label
        pos_label = pd.unique(target)[0]

        resROC_BinSim_PLSDA = metsta.PLSDA_ROC_cv(treated_data, target, # Data and target
                            pos_label=pos_label, # Positive label
                            n_comp=4, # Number of components
                            scale=False, # Set scale to True only if you did not do scaling in pre-treatments
                            n_iter=15, # Number of iterations to repeat 
                            cv=None, n_fold=3) # Method of cross-validation (None is stratified cv) and the number of folds
        
        with sns.axes_style("whitegrid"):
            with sns.plotting_context("notebook", font_scale=1.2):
                f, ax = plt.subplots(1, 1, figsize=(5,5), constrained_layout=True)
                res = resROC_BinSim_PLSDA
                mean_fpr = res['average fpr']
                mean_tpr = res['average tpr']
                mean_auc = res['mean AUC']
                mean_fpr = [0,] + list(mean_fpr)
                mean_tpr = [0,] + list(mean_tpr)
                ax.plot(mean_fpr, mean_tpr,
                       label=f'AUC = {mean_auc:.3f}',
                       lw=2, alpha=0.8)
                ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='lightgrey', alpha=.8)
                ax.legend()
                ax.set_xlim(None, 1)
                ax.set_ylim(0, None)
                ax.set(xlabel='False positive rate', ylabel='True positive rate')
                ax.set_title('PLS-DA ROC Curve (BinSim)', fontsize=15)

                #f.savefig('Name_BinSim_PLSDA_ROCcurve.jpg', dpi=400) # Save the figure
    else:
        print('Your target has more than 2 classes. Thus, ROC curves are not made.')

XGBoost

In [ ]:
if xgb_analysis:

    xgb_max_n_estimators_binsim = 300

    xgb_optim_params_binsim = {'n_estimators': range(10,xgb_max_n_estimators_binsim+1,5)} 

In [ ]:
if xgb_analysis:
    
    XGB_Optim_BinSim = metsta.optimise_xgb_parameters(bin_data, target, xgb_optim_params_binsim, regression, objective)

In [ ]:
if xgb_analysis:

    param_to_plot = 'n_estimators'

    # Plotting the results and adjusting parameters of the plot
    with sns.axes_style("whitegrid"):
        with sns.plotting_context("notebook", font_scale=1.2):
            f, ax = plt.subplots(1, 1, figsize=(6,6), constrained_layout=True) # Set Figure Size

            c_map = sns.color_palette('tab10', 10)

            ax.plot(XGB_Optim_BinSim.cv_results_['param_n_estimators'], [s*100 for s in XGB_Optim_BinSim.cv_results_['mean_test_score']])
            ax.set_ylabel('XGBoost CV Mean Accuracy (%)', fontsize=15) # Set the y_label and size
            ax.set_xlabel(param_to_plot, fontsize=15)
            ax.set_title('XGBoost', fontsize=18) # Set the title and size
            ax.set_ylim([30,101]) # Set the limits on the y axis

            #f.suptitle('Optimization of the number of trees')
            ax.legend(fontsize=15) # Set the legend and size
            plt.show()

In [ ]:
if xgb_analysis:

    n_estimators=300 # Number of trees in the model

    XGB_results_BinSim = metsta.XGB_model(bin_data, target, # Data and labels
                        regres=regression, obj=objective, # If you're doing regression or classificattion and objective function
                        return_cv=True, iter_num=5, # If you want cross calidation results and number of iterations for it
                        n_estimators=n_estimators, # Number of trees in the model
                        cv=None, n_fold=3, # Choose a method of cross-validation (None is stratified cv) and the nº of folds
            metrics = ('accuracy', 'f1_weighted', 'precision_weighted', 'recall_weighted')) # Choose the performance metrics

    XGB_results_summary_BinSim = pd.DataFrame(columns=['Value', 'Standard Deviation'])
    for k,v in XGB_results_BinSim.items():
        if k != 'model' and k != 'imp_feat':
            XGB_results_summary_BinSim.loc[k] = np.mean(v), np.std(v)

    print(XGB_results_summary_BinSim)

    imp_feats_xgb_BinSim = meta_data.copy()
    imp_feats_xgb_BinSim.insert(0,'Bucket label', imp_feats_xgb_BinSim.index)
    imp_feats_xgb_BinSim.insert(1,'Feature Importance', '')
    imp_feats_xgb_BinSim = imp_feats_xgb_BinSim.copy()
    for n in range(len(XGB_results_BinSim['imp_feat'])):
        imp_feats_xgb_BinSim['Feature Importance'].iloc[XGB_results_BinSim['imp_feat'][n][0]] = XGB_results_BinSim['imp_feat'][n][1]
    imp_feats_xgb_BinSim = imp_feats_xgb_BinSim.sort_values(by='Feature Importance', ascending=False)
    imp_feats_xgb_BinSim.index = range(1, len(imp_feats_xgb_BinSim)+1)
else:
    imp_feats_xgb_BinSim = "XGBoost Analysis was not performed"
    
imp_feats_xgb_BinSim

# Step 11: Find Specific Compounds <a class="anchor" id="step-11"></a>

**Back to [Table of Contents](#toc)**

This code will allow you to conveniently find specific compounds by their index, name, formula, or neutral mass / Probable _m/z_. The mass finders work even if you don't know the specific mass, as you can search only by the first few digits.

In [ ]:
id_type = 'name' #pick 'formula', 'name', 'Neutral Mass'/'Probable m/z', or 'index'
find_id = 'Palmitic acid'
if id_type == 'index':
    finder = processed_data.loc[processed_data.index.str.startswith(find_id)]
elif id_type in ['Neutral Mass','Probable m/z']:
    finder = processed_data.loc[processed_data[mass_val_col].str.startswith(find_id)]
elif id_type == 'formula':
    index_list = []
    for col in meta_cols_formulas:
        temp_df = processed_data[[col]].dropna()
        for t in temp_df.index:
            if find_id in temp_df[col][t]:
                index_list.append(t)
    if 'Formula' in processed_data.columns:
        temp_df = processed_data['Formula'].dropna()
        for t in temp_df.index:
            if find_id in temp_df[t]:
                index_list.append(t)
    finder = processed_data.loc[processed_data.index.isin(index_list)]
elif id_type == 'name':
    index_list = []
    for col in meta_cols_names:
        temp_df = processed_data[[col]].dropna()
        for t in temp_df.index:
            if find_id in temp_df[col][t]:
                index_list.append(t)
    if 'Name' in processed_data.columns:
        temp_df = processed_data['Name'].dropna()
        for t in temp_df.index:
            if find_id in temp_df[t]:
                index_list.append(t)
    finder = processed_data.loc[processed_data.index.isin(index_list)]
finder

In [ ]:
# Plot to see the intensities distribution (might become difficult to see with a lot of samples)
bar_plot_info = finder.replace({np.nan:0})
if len(bar_plot_info.index) == 1:
    fig, ax = plt.subplots(1,1, figsize=(16,6))
    x = np.arange(len(bar_plot_info.columns[-len(sample_cols):]))
    for comps in range(len(bar_plot_info.index)):
        ax.barh(x, np.array(bar_plot_info.iloc[comps, -len(sample_cols):]), color=sample_colours)
        ax.set_ylabel('Normalized Intensity', fontsize=15)
        ax.set_title(find_id, fontsize=15)
else:
    fig, axs = plt.subplots(1,len(bar_plot_info.index), figsize=(16,6))

    x = np.arange(len(bar_plot_info.columns[-len(sample_cols):]))
    for (comps, ax) in zip(range(len(bar_plot_info.index)), axs.ravel()):
        ax.barh(x, np.array(bar_plot_info.iloc[comps, -len(sample_cols):]), color=sample_colours)
        ax.set_yticks([])
        #ax.tick_params(labelleft=False)
        ax.set_ylabel('Normalized Intensity', fontsize=13)
        ax.set_title(find_id)
    ax = axs[0]
ax.set_yticks(x)
ax.set_yticklabels(bar_plot_info.columns[-len(sample_cols):], fontsize=12)
plt.show()

Search for the differences between groups

In [ ]:
# Calculate average intensities
gfinder = finder.copy()
for g in groups:
    gfinder[g+' Average'] = gfinder[gfinder.columns.intersection(groups[g])].mean(axis=1)
    gfinder[g+' std'] = gfinder[gfinder.columns.intersection(groups[g])].std(axis=1)
gfinder

In [ ]:
avg_cols = [col for col in gfinder.columns if 'Average' in col]
std_cols = [col for col in gfinder.columns if 'std' in col]
group_colours = [label_colours[lbl] for lbl in classes]
bar_plot_info = gfinder.replace({np.nan:0})
if len(bar_plot_info.index) == 1:
    fig, ax = plt.subplots(1,1, figsize=(10,7))
    x = np.arange(len(avg_cols))
    for comps in bar_plot_info.index:
        ax.bar(x, np.array(bar_plot_info.loc[comps, avg_cols]), color=group_colours, yerr=np.array(bar_plot_info.loc[comps, std_cols]), capsize=12)
        ax.set_ylabel('Normalized Intensity', fontsize=15)
        ax.set_title(find_id, fontsize=15)
        ax.set_xticks(x)
        ax.set_xticklabels(groups, fontsize=12)
else:
    fig, axs = plt.subplots(1,len(bar_plot_info.index), figsize=(16,6))

    x = np.arange(len(avg_cols))
    for (comps, ax) in zip(bar_plot_info.index, axs.ravel()):
        ax.bar(x, np.array(bar_plot_info.loc[comps, avg_cols]), color=group_colours, yerr=np.array(bar_plot_info.loc[comps, std_cols]), capsize=12)
        ax.set_yticks([])
        #ax.tick_params(labelleft=False)
        ax.set_ylabel('Normalized Intensity', fontsize=13)
        ax.set_title(find_id)
        ax.set_xticks(x)
        ax.set_xticklabels(groups)
    ax = axs[0]
plt.show()

In [ ]:
if len(finder.index) == 1:
    its_list = []
    for g in groups:
        its = finder[finder.columns.intersection(groups[g])].T.reset_index(drop= True)
        its_list.append(its)
    its_df = pd.concat(its_list, axis=1)
    its_df = its_df.set_axis(groups, axis=1)
    fig, ax = plt.subplots(1,1, figsize=(12,7))
    box1= ax.boxplot(its_df, labels=its_df.columns, patch_artist=True, medianprops=dict(color='black'))
    for patch, color, lbl in zip(box1['boxes'], colours, its_df.columns):
        patch.set_facecolor(color)
    ax.tick_params(axis="x", labelsize=25)
    ax.tick_params(axis="y", labelsize=20)
    ax.set_title(find_id, fontsize=25)
    plt.show()
else:
    fig, axs = plt.subplots(1,len(finder.index), figsize=(16,6))
    for (comps, ax) in zip(finder.index, axs.ravel()):
        comps_df = finder.loc[[comps]]
        its_list = []
        for g in groups:
            its = comps_df[comps_df.columns.intersection(groups[g])].T.reset_index(drop= True)
            its_list.append(its)
        its_df = pd.concat(its_list, axis=1)
        its_df = its_df.set_axis(groups, axis=1)
        box1= ax.boxplot(its_df, labels=its_df.columns, patch_artist=True, medianprops=dict(color='black'))
        for patch, color, lbl in zip(box1['boxes'], colours, its_df.columns):
            patch.set_facecolor(color)
        ax.set_title(find_id, fontsize=15)
    ax = axs[0]
plt.show()

### Notebook made by:

- FT-ICR-MS-Lisboa Laboratory Group